## AIHub Search Strange Source in Json

### Development Environment

In [ ]:
%pip install kss==3.7.3

KSS Argument Error: Restart Jupyter Kernel Runtime

In [ ]:
%pip install python-mecab-ko

KSS 3.7.3 matches python-mecab-ko

In [ ]:
%pip install pandas

In [7]:
import re
import os
import kss
import json
from mecab import MeCab
import pandas as pd
from glob import glob
from pprint import pprint

In [8]:
pwd

'D:\\AIHUB'

### Function

In [70]:
def sorted_list(path_list):
    path_list = sorted(path_list, reverse=False)
    path_list = sorted(path_list, key=len)
    
    return path_list

In [71]:
def json_file_name_list(path_list):
    
    file_name  = [glob(i, recursive = True) for i in path_list][0]
    file_name = sorted_list(file_name)
    
    return file_name

In [72]:
def train_valid_json_file_name_list(path_list):

  train_file_name, valid_file_name = [glob(i, recursive = True) if 'rain' in i
                                      else glob(i, recursive = True)
                                      for i in path_list]

  train_file_name = sorted_list(train_file_name)
  valid_file_name = sorted_list(valid_file_name)
    
  return train_file_name, valid_file_name

In [73]:
def divide_source_file_list(l, n): 
    
  for i in range(0, len(l), n): 
    yield l[i:i + n] 

In [9]:
def formal_preprocessing_text(source):
    preprocessing_sentence_list = []
    
    source = source.strip()
    # strip으로 앞뒤 공백 제거

    source = re.sub(r"\[.*?\]|\{.*?\}", "", source)
    # 기타 괄호 제거할 시 괄호 내부에 모든 텍스트 제거


    try:
        bracket_form = re.compile('\(([^)]+)')
        text_in_small_bracket = bracket_form.findall(source)
    
    
        if type(text_in_small_bracket) == str:

            text = text_in_small_bracket

            text_size = len(text)
            last_index = source.find(text) + len(text)
            if len(source) >= last_index+1 and source[last_index-text_size-1] == '(' and source[last_index+1] == '.':
                source = source.replace(source[last_index-text_size-1 : last_index+1] + ".", ".")

            if len(text.split()) > 5 and bool(re.match(r'[.]|[!]|[?]', text[-1])) == True:
                small_bracket = "(" + text + ")"
                source = source.replace(small_bracket, text)    

        elif type(text_in_small_bracket) == list:

            for text in text_in_small_bracket:

                text_size = len(text)
                last_index = source.find(text) + len(text)
                if len(source) >= last_index+1 and source[last_index-text_size-1] == '(' and source[last_index+1] == '.':
                    source = source.replace(source[last_index-text_size-1 : last_index+1] + ".", ".")

                if len(text.split()) > 5 and bool(re.match(r'[.]|[!]|[?]', text[-1])) == True:
                    small_bracket = "(" + text + ")"
                    source = source.replace(small_bracket, text)    

    except:
        pass

        # 마침표(.) 앞에 소괄호')'가 있을시 소괄호 제거와 함께 소괄호 내부 텍스트 제거
        # 소괄호 내부 텍스트가 5어절 이상이고 끝이 온점(.). 느낌표(!). 물음표(?)일 떼 소괄호 제거
        
    
    if bool(re.match(r'[가나다라마바사아자차카타파하]+[.]', source[:2])) == True:
        source = re.sub(source[:2], "", source)
        
    source = re.sub(r' [가나다라마바사아자차카타파하]+[.]', "", source)
    # '가.', '나.', ... 형태의 문자열 제거 
        
    for sentence in kss.split_sentences(source, use_heuristic=False,
                                        num_workers=32):
    # KSS(Korean Sentence Segmentation)로 문장 분리 
    # Formal articles (wiki, news, essays): recommend to False
    

        if re.search("^[A-Za-z0-9ㄱ-ㅎ가-힣一-鿕㐀-䶵豈-龎]", sentence[0]) is not None and \
            bool(re.match(r'[.]|[!]|[?]', sentence[-1])) == True and \
            len(sentence.split()) > 5:
            # 문장의 시작이 특수문자인 문장(영어 대소문자, 한글, 한자, 숫자, -, + 제외
            # 문장의 끝이 온점(.). 느낌표(!). 물음표(?)가 아닌 문장 제외
            # 다섯 어절 이하 문장 제외


            if ']' in sentence and '[' not in sentence:
                sentence  = re.sub(r".*?]", "", sentence)    
            # 중괄호 앞에 있는 '성명/직함]' 형태 제거


            sentence = re.sub(r"[^A-Za-z0-9ㄱ-ㅎ가-힣一-鿕㐀-䶵豈-龎()+-.,]", " ", sentence)
            # 특수문자 제거(영어 대소문자, 한글, 한자, 숫자, -, +, 소괄호, 마침표, 쉼표, 제외)

            sentence = sentence.strip()
            # strip으로 앞뒤 공백 제거

            total_length = len(sentence.replace(" " , ""))
            hangeul_length = len(re.sub(r"[^ㄱ-ㅣ가-힣\s]", "", sentence.replace(" " , "")))
            hangeul_ratio = hangeul_length / total_length
            if hangeul_ratio >= 0.5:
            # 한글이 아닌 문자열이 50% 이상이 넘은 문장 제외

                for sentence2 in kss.split_sentences(sentence, use_heuristic=False,
                                        num_workers=32):
                    for sentence3 in kss.split_sentences(sentence2, use_heuristic=False,
                                                         num_workers=32):
                        preprocessing_sentence_list.append(sentence3)

            # 마지막에 KSS(Korean Sentence Segmentation)로 문장 분리 2번 실행

  
    return preprocessing_sentence_list

In [1]:
def legal_preprocessing_text(source):
    preprocessing_sentence_list = []
    source = source.strip()
    # strip으로 앞뒤 공백 제거

    source = re.sub(r"\[.*?\]|\{.*?\}", "", source)
    # 기타 괄호 제거할 시 괄호 내부에 모든 텍스트 제거

    try:
        bracket_form = re.compile('\(([^)]+)')
        text_in_small_bracket = bracket_form.findall(source)
    
    
        if type(text_in_small_bracket) == str:

            text = text_in_small_bracket

            text_size = len(text)
            last_index = source.find(text) + len(text)
            if len(source) >= last_index+1 and source[last_index-text_size-1] == '(' and source[last_index+1] == '.':
                source = source.replace(source[last_index-text_size-1 : last_index+1] + ".", ".")

            if len(text.split()) > 5 and bool(re.match(r'[.]|[!]|[?]', text[-1])) == True:
                small_bracket = "(" + text + ")"
                source = source.replace(small_bracket, text)    

        elif type(text_in_small_bracket) == list:

            for text in text_in_small_bracket:

                text_size = len(text)
                last_index = source.find(text) + len(text)
                if len(source) >= last_index+1 and source[last_index-text_size-1] == '(' and source[last_index+1] == '.':
                    source = source.replace(source[last_index-text_size-1 : last_index+1] + ".", ".")

                if len(text.split()) > 5 and bool(re.match(r'[.]|[!]|[?]', text[-1])) == True:
                    small_bracket = "(" + text + ")"
                    source = source.replace(small_bracket, text)    

    except:
        pass

        # 마침표(.) 앞에 소괄호')'가 있을시 소괄호 제거와 함께 소괄호 내부 텍스트 제거
        # 소괄호 내부 텍스트가 5어절 이상이고 끝이 온점(.). 느낌표(!). 물음표(?)일 떼 소괄호 제거
        
    
    if bool(re.match(r'[가나다라마바사아자차카타파하]+[.]', source[:2])) == True:
        source = re.sub(source[:2], "", source)
        
    source = re.sub(r' [가나다라마바사아자차카타파하]+[.]', "", source)
    # '가.', '나.', ... 형태의 문자열 제거 
 
    for sentence in kss.split_sentences(source, use_heuristic=False,
                                        num_workers=32):
    # KSS(Korean Sentence Segmentation)로 문장 분리 
    # Formal articles (wiki, news, essays): recommend to False
    
    
    source = re.sub(r'제\d{1,}조의\d{1,}\([^)]+\)', "", source)
    source = re.sub(r'제\d{1,}조\([^)]+\)', "",source)
    sources = re.split(r"[0-9]+[.]|①|②|③|④|⑤|⑥|⑦|⑧|⑨|⑩|⑪|⑫|⑬|⑭|⑮", source)
    # '제n조의n항(text)', '제n조(text)' 꼴 제거
    # 숫자.(1. 2. ...), 원숫자(①, ②, ③, ...)를 기준으로 문장 분리
    
    for source in sources:
    
        for sentence in kss.split_sentences(source, use_heuristic=False,
                                            num_workers=32):
        # KSS(Korean Sentence Segmentation)로 문장 분리 
        # Formal articles (wiki, news, essays): recommend to False

            if re.search("^[A-Za-z0-9ㄱ-ㅎ가-힣一-鿕㐀-䶵豈-龎①②③④⑤⑥⑦⑧⑨⑩⑪⑫⑬⑭⑮]", sentence[0]) is not None and \
                bool(re.match(r'[.]|[!]|[?]', sentence[-1])) == True and \
                len(sentence.split()) > 5:
                # 문장의 시작이 특수문자인 문장(영어 대소문자, 한글, 한자, 숫자, -, +, 
                # ①, ②, ③, ④, ⑤, ⑥, ⑦, ⑧, ⑨, ⑩, ⑪, ⑫, ⑬, ⑭, ⑮ 제외) 제외
                # 문장의 끝이 온점(.). 느낌표(!). 물음표(?)가 아닌 문장 제외
                # 다섯 어절 이하 문장 제외

                sentence = re.sub(r"[^A-Za-z0-9ㄱ-ㅎ가-힣一-鿕㐀-䶵豈-龎()+-.,]", " ", sentence)
                # 특수문자 제거(영어 대소문자, 한글, 한자, 숫자, -, +, 소괄호, 마침표, 쉼표, 제외)

                sentence = sentence.strip()
                # strip으로 앞뒤 공백 제거

                total_length = len(sentence.replace(" " , ""))
                hangeul_length = len(re.sub(r"[^ㄱ-ㅣ가-힣\s]", "", sentence.replace(" " , "")))
                hangeul_ratio = hangeul_length / total_length
                if hangeul_ratio >= 0.5:
                # 한글이 아닌 문자열이 50% 이상이 넘은 문장 제외
                    for sentence2 in kss.split_sentences(sentence, use_heuristic=False,
                                            num_workers=32):
                        for sentence3 in kss.split_sentences(sentence2, use_heuristic=False,
                                                             num_workers=32):
                            preprocessing_sentence_list.append(sentence3)

                # 마지막에 KSS(Korean Sentence Segmentation)로 문장 분리 2번 실행

    return preprocessing_sentence_list

In [3]:
def tableqa_preprocessing_text(source):
    
    preprocessing_sentence_list = []
    source_split = re.split('br|tr|td|table|tbody|rowspan', source)
    # 문자열 'br', 'tr', 'td', 'table', 'tbody'를 기준으로 문장 분리

    for source in source_split:
        sentence = re.sub('(br|tr|td|table|tbody|rowspan|<|>|/)', '', source)
        # 문자열 'br', 'tr', 'td', 'table', 'tbody' 특수기호 '<', '>', '/'를 제외
            
        sentence = sentence.strip()
        # strip으로 앞뒤 공백 제거
        
        sentence = re.sub(r"\[.*?\]|\{.*?\}", "", sentence)
        # 기타 괄호 제거할 시 괄호 내부에 모든 텍스트 제거

        bracket_form = re.compile('\(([^)]+)')
        text_in_small_bracket = bracket_form.findall(sentence)

    try:
        bracket_form = re.compile('\(([^)]+)')
        text_in_small_bracket = bracket_form.findall(sentence)
    
    
        if type(text_in_small_bracket) == str:

            text = text_in_small_bracket

            text_size = len(text)
            last_index = sentence.find(text) + len(text)
            if len(sentence) >= last_index+1 and sentence[last_index-text_size-1] == '(' and sentence[last_index+1] == '.':
                sentence = sentence.replace(sentence[last_index-text_size-1 : last_index+1] + ".", ".")

            if len(text.split()) > 5 and bool(re.match(r'[.]|[!]|[?]', text[-1])) == True:
                small_bracket = "(" + text + ")"
                sentence = sentence.replace(small_bracket, text)    

        elif type(text_in_small_bracket) == list:

            for text in text_in_small_bracket:

                text_size = len(text)
                last_index = sentence.find(text) + len(text)
                if len(sentence) >= last_index+1 and sentence[last_index-text_size-1] == '(' and sentence[last_index+1] == '.':
                    sentence = sentence.replace(sentence[last_index-text_size-1 : last_index+1] + ".", ".")

                if len(text.split()) > 5 and bool(re.match(r'[.]|[!]|[?]', text[-1])) == True:
                    small_bracket = "(" + text + ")"
                    sentence = sentence.replace(small_bracket, text)    

    except:
        pass

        # 마침표(.) 앞에 소괄호')'가 있을시 소괄호 제거와 함께 소괄호 내부 텍스트 제거
        # 소괄호 내부 텍스트가 5어절 이상이고 끝이 온점(.). 느낌표(!). 물음표(?)일 떼 소괄호 제거
        
    
    if bool(re.match(r'[가나다라마바사아자차카타파하]+[.]', sentence[:2])) == True:
        sentence = re.sub(sentence[:2], "", sentence)
        
    sentence = re.sub(r' [가나다라마바사아자차카타파하]+[.]', "", sentence)
    # '가.', '나.', ... 형태의 문자열 제거 
        

        if len(sentence) > 1 and \
        re.search("^[A-Za-z0-9ㄱ-ㅎ가-힣一-鿕㐀-䶵豈-龎]", sentence[0]) is not None and \
        bool(re.match(r'[.]|[!]|[?]', sentence[-1])) == True and  \
        len(sentence.split()) > 5:
         # 문장의 시작이 특수문자인 문장(영어 대소문자, 한글, 한자, 숫자, -, +, 
        # 문장의 끝이 온점(.). 느낌표(!). 물음표(?)가 아닌 문장 제외
        # 다섯 어절 이하 문장 제외
        
            
            sentence = re.sub(r"[^A-Za-z0-9ㄱ-ㅎ가-힣一-鿕㐀-䶵豈-龎()+-.,]", " ", sentence)
            # 특수문자 제거(영어 대소문자, 한글, 한자, 숫자, -, +, 소괄호, 마침표, 쉼표 제외)

            sentence = sentence.strip()
            # strip으로 앞뒤 공백 제거
            
            total_length = len(sentence.replace(" " , ""))
            hangeul_length = len(re.sub(r"[^ㄱ-ㅣ가-힣\s]", "", sentence.replace(" " , "")))
            hangeul_ratio = hangeul_length / total_length
            if hangeul_ratio >= 0.5:
            # 한글이 아닌 문자열이 50% 이상이 넘은 문장 제외
                for sentence2 in kss.split_sentences(sentence, use_heuristic=False,
                                        num_workers=32):
                    for sentence3 in kss.split_sentences(sentence2, use_heuristic=False,
                                                         num_workers=32):
                        preprocessing_sentence_list.append(sentence3)

            # 마지막에 KSS(Korean Sentence Segmentation)로 문장 분리 2번 실행


    return preprocessing_sentence_list

### AIHUB 전문분야 말뭉치

[Source](https://www.aihub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100&aihubDataSe=realm&dataSetSn=110)

In [618]:
path_list = ['AIHUB_전문분야 말뭉치/Training/'+ '/**/*.json', 
             'AIHUB_전문분야 말뭉치/Validation/'+ '/**/*.json']
train_file_name, valid_file_name = train_valid_json_file_name_list(path_list)

In [452]:
source_file = train_file_name[1]
print(source_file)

with open(source_file, 'r', encoding='utf-8') as one_json_file:
    one_json_sample = json.load(one_json_file)

if '논문_' in source_file:
    source_list = list(pd.DataFrame(one_json_sample['data'])['text'])

    
elif '특허_z' in source_file:
    source_df = pd.DataFrame(one_json_sample['data'])
    source_list = list(source_df[source_df['attr'] == 'abs']['text'])


elif '특허_0' in source_file or '특허_1' in source_file:
    source_df = pd.DataFrame(one_json_sample['data'])
    source_dict = dict(source_df[source_df['attr'] == '초록']['sentence'].explode())
    source_json = pd.json_normalize(source_dict)
    source_list = list(source_json.filter(regex='text').values[0])
    

elif '법령' in source_file or '판례' in source_file:
    source_df = pd.DataFrame(one_json_sample['data'])
    source_dict = dict(source_df['sentence'].explode())
    source_json = pd.json_normalize(source_dict)
    source_list = list(source_json.filter(regex='text').values[0])

AIHUB_전문분야 말뭉치/Training\[원천]전문분야_train\특허_02.json


In [453]:
print(source_list[2190])

본 발명의 실시 예에 따른 유기 발광 디스플레이 장치의 구동 방법은, 유기 발광 다이오드를 발광시키는 화소 회로로 구성된 복수의 화소를 포함 디스플레이 패널과, 상기 디스플레이 패널을 구동시키는 구동 회로부를 포함하는 유기 발광 디스플레이 장치의 구동 방법에 있어서, 입력된 데이터를 카운팅하여 누적 데이터를 생성하는 단계； 초기 모델링 데이터를 이용하여 상기 누적 데이터에 따른 드라이빙 TFT의 특성 변화를 예측하는 단계； 상기 드라이빙 TFT의 특성 변화를 예측에 따라 보상 데이터를 생성하고, 상기 보상 데이터를 이용하여 상기 드라이빙 TFT의 특성 변화를 보상하는 단계； 및 파워가 온되는 시점, 파워가 오프되는 시점 또는 사용자 선택에 의한 보상 신호가 입력될 때, 상기 전체 화소의 드라이빙 TFT의 특성을 센싱하여 센싱 데이터를 생성하고, 상기 센싱 데이터에 따라 보상 데이터를 생성하여 상기 전체 드라이빙 TFT의 특성 변화를 보상하는 단계를 포함한다.


In [454]:
preprocessing_sentences = formal_preprocessing_text(source_list[2190])
for preprocessing_sentence in preprocessing_sentences:
    print(preprocessing_sentence, end="\n\n")

본 발명의 실시 예에 따른 유기 발광 디스플레이 장치의 구동 방법은, 유기 발광 다이오드를 발광시키는 화소 회로로 구성된 복수의 화소를 포함 디스플레이 패널과, 상기 디스플레이 패널을 구동시키는 구동 회로부를 포함하는 유기 발광 디스플레이 장치의 구동 방법에 있어서, 입력된 데이터를 카운팅하여 누적 데이터를 생성하는 단계 초기 모델링 데이터를 이용하여 상기 누적 데이터에 따른 드라이빙 TFT의 특성 변화를 예측하는 단계 상기 드라이빙 TFT의 특성 변화를 예측에 따라 보상 데이터를 생성하고, 상기 보상 데이터를 이용하여 상기 드라이빙 TFT의 특성 변화를 보상하는 단계 및 파워가 온되는 시점, 파워가 오프되는 시점 또는 사용자 선택에 의한 보상 신호가 입력될 때, 상기 전체 화소의 드라이빙 TFT의 특성을 센싱하여 센싱 데이터를 생성하고, 상기 센싱 데이터에 따라 보상 데이터를 생성하여 상기 전체 드라이빙 TFT의 특성 변화를 보상하는 단계를 포함한다.



In [455]:
source_file = train_file_name[10]
print(source_file)

with open(source_file, 'r', encoding='utf-8') as one_json_file:
    one_json_sample = json.load(one_json_file)

if '논문_' in source_file:
    source_list = list(pd.DataFrame(one_json_sample['data'])['text'])

    
elif '특허_z' in source_file:
    source_df = pd.DataFrame(one_json_sample['data'])
    source_list = list(source_df[source_df['attr'] == 'abs']['text'])


elif '특허_0' in source_file or '특허_1' in source_file:
    source_df = pd.DataFrame(one_json_sample['data'])
    source_dict = dict(source_df[source_df['attr'] == '초록']['sentence'].explode())
    source_json = pd.json_normalize(source_dict)
    source_list = list(source_json.filter(regex='text').values[0])
    

elif '법령' in source_file or '판례' in source_file:
    source_df = pd.DataFrame(one_json_sample['data'])
    source_dict = dict(source_df['sentence'].explode())
    source_json = pd.json_normalize(source_dict)
    source_list = list(source_json.filter(regex='text').values[0])

AIHUB_전문분야 말뭉치/Training\[원천]전문분야_train\특허_11.json


In [456]:
print(source_list[330])

본원 발명의 페놀 수지 발포체는, 페놀 수지와, 염소화 하이드로플루오로올레핀, 비염소화 하이드로플루오로올레핀, 및 할로겐화탄화수소로 이루어지는 군에서 선택되는 적어도 1 종을 함유하고, 밀도가 20 ㎏/㎥ 이상 100 ㎏/㎥ 이하이고, 평균 기포 직경이 10 ㎛ 이상 300 ㎛ 이하이고, 독립 기포율이 80 ％ 이상 99 ％ 이하이고, 투습률이 0.38 ng/(m・s・㎩) 이상 2.00 ng/(m・s・㎩) 이하인 것을 특징으로 한다.


In [457]:
preprocessing_sentences = formal_preprocessing_text(source_list[330])
for preprocessing_sentence in preprocessing_sentences:
    print(preprocessing_sentence, end="\n\n")

본원 발명의 페놀 수지 발포체는, 페놀 수지와, 염소화 하이드로플루오로올레핀, 비염소화 하이드로플루오로올레핀, 및 할로겐화탄화수소로 이루어지는 군에서 선택되는 적어도 1 종을 함유하고, 밀도가 20 이상 100 이하이고, 평균 기포 직경이 10 이상 300 이하이고, 독립 기포율이 80 이상 99 이하이고, 투습률이 0.38 ng (m s ) 이상 2.00 ng (m s ) 이하인 것을 특징으로 한다.



In [458]:
source_file = train_file_name[24]
print(source_file)

with open(source_file, 'r', encoding='utf-8') as one_json_file:
    one_json_sample = json.load(one_json_file)

if '논문_' in source_file:
    source_list = list(pd.DataFrame(one_json_sample['data'])['text'])

    
elif '특허_z' in source_file:
    source_df = pd.DataFrame(one_json_sample['data'])
    source_list = list(source_df[source_df['attr'] == 'abs']['text'])


elif '특허_0' in source_file or '특허_1' in source_file:
    source_df = pd.DataFrame(one_json_sample['data'])
    source_dict = dict(source_df[source_df['attr'] == '초록']['sentence'].explode())
    source_json = pd.json_normalize(source_dict)
    source_list = list(source_json.filter(regex='text').values[0])
    

elif '법령' in source_file or '판례' in source_file:
    source_df = pd.DataFrame(one_json_sample['data'])
    source_dict = dict(source_df['sentence'].explode())
    source_json = pd.json_normalize(source_dict)
    source_list = list(source_json.filter(regex='text').values[0])

AIHUB_전문분야 말뭉치/Training\[원천]전문분야_train\판례_case03.json


In [459]:
print(source_list[7])

이것은 명함 본래의 기능에 부합하지 아니할 뿐만 아니라, 선거운동 기회균등의 원칙에 반하고, 예비후보자의 선거운동의 강화에만 치우친 나머지, 배우자의 유무라는 우연적인 사정에 근거하여 합리적 이유 없이 배우자 없는 예비후보자를 차별 취급하는 것이므로, 이 사건 3호 법률조항은 청구인의 평등권을 침해한다.


In [460]:
preprocessing_sentences = formal_preprocessing_text(source_list[7])
for preprocessing_sentence in preprocessing_sentences:
    print(preprocessing_sentence, end="\n\n")

이것은 명함 본래의 기능에 부합하지 아니할 뿐만 아니라, 선거운동 기회균등의 원칙에 반하고, 예비후보자의 선거운동의 강화에만 치우친 나머지, 배우자의 유무라는 우연적인 사정에 근거하여 합리적 이유 없이 배우자 없는 예비후보자를 차별 취급하는 것이므로, 이 사건 3호 법률조항은 청구인의 평등권을 침해한다.



In [461]:
source_file = train_file_name[2]
print(source_file)

with open(source_file, 'r', encoding='utf-8') as one_json_file:
    one_json_sample = json.load(one_json_file)

if '논문_' in source_file:
    source_list = list(pd.DataFrame(one_json_sample['data'])['text'])

    
elif '특허_z' in source_file:
    source_df = pd.DataFrame(one_json_sample['data'])
    source_list = list(source_df[source_df['attr'] == 'abs']['text'])


elif '특허_0' in source_file or '특허_1' in source_file:
    source_df = pd.DataFrame(one_json_sample['data'])
    source_dict = dict(source_df[source_df['attr'] == '초록']['sentence'].explode())
    source_json = pd.json_normalize(source_dict)
    source_list = list(source_json.filter(regex='text').values[0])
    

elif '법령' in source_file or '판례' in source_file:
    source_df = pd.DataFrame(one_json_sample['data'])
    source_dict = dict(source_df['sentence'].explode())
    source_json = pd.json_normalize(source_dict)
    source_list = list(source_json.filter(regex='text').values[0])

AIHUB_전문분야 말뭉치/Training\[원천]전문분야_train\특허_03.json


In [462]:
print(source_list[12])

본 발명의 로프는 트위스트 얀의 적어도 일부, 바람직하게는 전체 트위스트 얀의 섬도가 각 경우 최대 5000dtex이고, 트위스트 얀의 적어도 일부, 바람직하게는 전체 트위스트 얀의 트위스트 레이트가 각 경우 적어도 150T/m인 것을 특징으로 한다.


In [463]:
preprocessing_sentences = formal_preprocessing_text(source_list[12])
for preprocessing_sentence in preprocessing_sentences:
    print(preprocessing_sentence, end="\n\n")

본 발명의 로프는 트위스트 얀의 적어도 일부, 바람직하게는 전체 트위스트 얀의 섬도가 각 경우 최대 5000dtex이고, 트위스트 얀의 적어도 일부, 바람직하게는 전체 트위스트 얀의 트위스트 레이트가 각 경우 적어도 150T m인 것을 특징으로 한다.



In [464]:
print(source_list[116])

6 자유도(6-DOF) 관성 측정 시스템의 디바이스 층은 x-y 평면에 형성된단일 프루프 매스 6축 관성 센서를 포함하며, 관성 센서는 단일의 중앙 앵커의 주위에 현가된메인 프루프 매스 부분을 포함하고, 메인프루프 매스 부분은 관성 센서의 에지부를 향해 바깥쪽으로 연장된 방사형 부분을 포함하며, 관성 센서는 또한 단일의 중앙 앵커로부터6축 관성 센서를 현가하도록 구성된 중앙 서스펜션 시스템과 가동 부분 및 정지 부분을 구비하는 구동 전극을 포함하며, 가동 부분은 방사형 부분에 결합되고, 구동 전극과 중앙 서스펜션 시스템은 6축 관성 센서를 x-y 평면에 대해 법선 방향인 z축을 중심으로 구동 주파수로 진동시키도록 구성된다.


In [465]:
preprocessing_sentences = formal_preprocessing_text(source_list[116])
for preprocessing_sentence in preprocessing_sentences:
    print(preprocessing_sentence, end="\n\n")

6 자유도(6-DOF) 관성 측정 시스템의 디바이스 층은 x-y 평면에 형성된단일 프루프 매스 6축 관성 센서를 포함하며, 관성 센서는 단일의 중앙 앵커의 주위에 현가된메인 프루프 매스 부분을 포함하고, 메인프루프 매스 부분은 관성 센서의 에지부를 향해 바깥쪽으로 연장된 방사형 부분을 포함하며, 관성 센서는 또한 단일의 중앙 앵커로부터6축 관성 센서를 현가하도록 구성된 중앙 서스펜션 시스템과 가동 부분 및 정지 부분을 구비하는 구동 전극을 포함하며, 가동 부분은 방사형 부분에 결합되고, 구동 전극과 중앙 서스펜션 시스템은 6축 관성 센서를 x-y 평면에 대해 법선 방향인 z축을 중심으로 구동 주파수로 진동시키도록 구성된다.



In [466]:
print(source_list[800])

전극체(2)는 시트 형상의 정극(5) 및 부극(4)을 세퍼레이터(6)를 개재해서 편평 형상으로 권회해서 이루어지고, 전지 용기(1) 내에, 권회에 의해 얻어지는 만곡부가 덮개측과, 저면측에 위치하도록 수용된다.


In [467]:
preprocessing_sentences = formal_preprocessing_text(source_list[800])
for preprocessing_sentence in preprocessing_sentences:
    print(preprocessing_sentence, end="\n\n")

전극체(2)는 시트 형상의 정극(5) 및 부극(4)을 세퍼레이터(6)를 개재해서 편평 형상으로 권회해서 이루어지고, 전지 용기(1) 내에, 권회에 의해 얻어지는 만곡부가 덮개측과, 저면측에 위치하도록 수용된다.



In [501]:
source_file = train_file_name[30]
print(source_file)

with open(source_file, 'r', encoding='utf-8') as one_json_file:
    one_json_sample = json.load(one_json_file)

if '논문_' in source_file:
    source_list = list(pd.DataFrame(one_json_sample['data'])['text'])

    
elif '특허_z' in source_file:
    source_df = pd.DataFrame(one_json_sample['data'])
    source_list = list(source_df[source_df['attr'] == 'abs']['text'])


elif '특허_0' in source_file or '특허_1' in source_file:
    source_df = pd.DataFrame(one_json_sample['data'])
    source_dict = dict(source_df[source_df['attr'] == '초록']['sentence'].explode())
    source_json = pd.json_normalize(source_dict)
    source_list = list(source_json.filter(regex='text').values[0])
    

elif '법령' in source_file or '판례' in source_file:
    source_df = pd.DataFrame(one_json_sample['data'])
    source_dict = dict(source_df['sentence'].explode())
    source_json = pd.json_normalize(source_dict)
    source_list = list(source_json.filter(regex='text').values[0])

AIHUB_전문분야 말뭉치/Training\[원천]전문분야_train\법령_l_1302_1.json


In [502]:
print(source_list[92])

제19조의2(노무법인의 설립인가신청) ① 법 제7조의4제1항 전단에 따른 노무법인의 설립인가를 받으려는 자는 고용노동부령으로 정하는 바에 따라 노무법인의 설립인가신청서에 다음 각 호의 서류를 첨부하여 고용노동부장관에게 제출하여야 한다. 1. 정관 2. 업무계획서 및 예산서 3. 그 밖에 고용노동부장관이 정하는 서류 ② 고용노동부장관은 법 제7조의4제1항 전단에 따른 노무법인의 설립인가를 하였을 때에는 노무법인 인가대장에 다음 각 호의 사항을 기록하고 신청인에게 노무법인 설립인가증을 발급하여야 한다. 1. 인가번호 및 인가연월일 2. 노무법인의 명칭 3. 주사무소 및 분사무소의 소재지 4. 사원의 성명 및 주소 5. 그 밖에 고용노동부장관이 필요하다고 인정하는 사항 ③ 고용노동부장관은 제2항에 따라 신청인에게 노무법인 설립인가증을 발급한 경우에는 그 사실을 공인노무사회에 알려주어야 한다. [전문개정 2010·11·19]


In [504]:
preprocessing_sentences = legal_preprocessing_text(source_list[92])
for preprocessing_sentence in preprocessing_sentences:
    print(preprocessing_sentence, end="\n\n")

법 제7조의4제1항 전단에 따른 노무법인의 설립인가를 받으려는 자는 고용노동부령으로 정하는 바에 따라 노무법인의 설립인가신청서에 다음 각 호의 서류를 첨부하여 고용노동부장관에게 제출하여야 한다.

고용노동부장관은 법 제7조의4제1항 전단에 따른 노무법인의 설립인가를 하였을 때에는 노무법인 인가대장에 다음 각 호의 사항을 기록하고 신청인에게 노무법인 설립인가증을 발급하여야 한다.

고용노동부장관은 제2항에 따라 신청인에게 노무법인 설립인가증을 발급한 경우에는 그 사실을 공인노무사회에 알려주어야 한다.



In [505]:
source_list[406]

'제5조(변경등기) 제2조 각 호의 사항이 변경되었을 때에는 주된 사무소의 소재지에서는 2주일 이내에, 지점의 소재지에서는 3주일 이내에 각각 해당 변경사항을 등기하여야 한다. [전문개정 2010·10·13]'

In [506]:
preprocessing_sentences = legal_preprocessing_text(source_list[406])
for preprocessing_sentence in preprocessing_sentences:
    print(preprocessing_sentence, end="\n\n")

제2조 각 호의 사항이 변경되었을 때에는 주된 사무소의 소재지에서는 2주일 이내에, 지점의 소재지에서는 3주일 이내에 각각 해당 변경사항을 등기하여야 한다.



### AIHUB 대규모 웹데이터 기반 한국어 말뭉치 데이터

[Source](https://www.aihub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100&aihubDataSe=realm&dataSetSn=624)

In [507]:
path_list = ['AIHUB_웹데이터 기반 한국어 말뭉치 데이터/Training/원천데이터/TS1/IT_과학/' + '**/*.json',
'AIHUB_웹데이터 기반 한국어 말뭉치 데이터/Training/원천데이터/TS1/건강/' + '**/*.json',
'AIHUB_웹데이터 기반 한국어 말뭉치 데이터/Training/원천데이터/TS1/경제/' + '**/*.json',
'AIHUB_웹데이터 기반 한국어 말뭉치 데이터/Training/원천데이터/TS1/교육/' + '**/*.json',
'AIHUB_웹데이터 기반 한국어 말뭉치 데이터/Training/원천데이터/TS1/국제/' + '**/*.json',
'AIHUB_웹데이터 기반 한국어 말뭉치 데이터/Training/원천데이터/TS1/라이프스타일/' + '**/*.json',
'AIHUB_웹데이터 기반 한국어 말뭉치 데이터/Training/원천데이터/TS1/문화/' + '**/*.json',
'AIHUB_웹데이터 기반 한국어 말뭉치 데이터/Training/원천데이터/TS1/사건사고/' + '**/*.json',
'AIHUB_웹데이터 기반 한국어 말뭉치 데이터/Training/원천데이터/TS1/사회일반/' + '**/*.json',
'AIHUB_웹데이터 기반 한국어 말뭉치 데이터/Training/원천데이터/TS1/산업/' + '**/*.json',
'AIHUB_웹데이터 기반 한국어 말뭉치 데이터/Training/원천데이터/TS1/스포츠/' + '**/*.json',
'AIHUB_웹데이터 기반 한국어 말뭉치 데이터/Training/원천데이터/TS1/여성복지/' + '**/*.json',
'AIHUB_웹데이터 기반 한국어 말뭉치 데이터/Training/원천데이터/TS1/여행레저/' + '**/*.json',
'AIHUB_웹데이터 기반 한국어 말뭉치 데이터/Training/원천데이터/TS1/연예/' + '**/*.json',
'AIHUB_웹데이터 기반 한국어 말뭉치 데이터/Training/원천데이터/TS1/정치/' + '**/*.json',
'AIHUB_웹데이터 기반 한국어 말뭉치 데이터/Training/원천데이터/TS1/지역/' + '**/*.json',
'AIHUB_웹데이터 기반 한국어 말뭉치 데이터/Training/원천데이터/TS1/취미/' + '**/*.json',
'AIHUB_웹데이터 기반 한국어 말뭉치 데이터/Validation/원천데이터/VS1/IT_과학/' + '**/*.json',
'AIHUB_웹데이터 기반 한국어 말뭉치 데이터/Validation/원천데이터/VS1/건강/' + '**/*.json',
'AIHUB_웹데이터 기반 한국어 말뭉치 데이터/Validation/원천데이터/VS1/경제/' + '**/*.json',
'AIHUB_웹데이터 기반 한국어 말뭉치 데이터/Validation/원천데이터/VS1/교육/' + '**/*.json',
'AIHUB_웹데이터 기반 한국어 말뭉치 데이터/Validation/원천데이터/VS1/국제/' + '**/*.json',
'AIHUB_웹데이터 기반 한국어 말뭉치 데이터/Validation/원천데이터/VS1/라이프스타일/' + '**/*.json',
'AIHUB_웹데이터 기반 한국어 말뭉치 데이터/Validation/원천데이터/VS1/문화/' + '**/*.json',
'AIHUB_웹데이터 기반 한국어 말뭉치 데이터/Validation/원천데이터/VS1/사건사고/' + '**/*.json',
'AIHUB_웹데이터 기반 한국어 말뭉치 데이터/Validation/원천데이터/VS1/사회일반/' + '**/*.json',
'AIHUB_웹데이터 기반 한국어 말뭉치 데이터/Validation/원천데이터/VS1/산업/' + '**/*.json',
'AIHUB_웹데이터 기반 한국어 말뭉치 데이터/Validation/원천데이터/VS1/스포츠/' + '**/*.json',
'AIHUB_웹데이터 기반 한국어 말뭉치 데이터/Validation/원천데이터/VS1/여성복지/' + '**/*.json',
'AIHUB_웹데이터 기반 한국어 말뭉치 데이터/Validation/원천데이터/VS1/여행레저/' + '**/*.json',
'AIHUB_웹데이터 기반 한국어 말뭉치 데이터/Validation/원천데이터/VS1/연예/' + '**/*.json',
'AIHUB_웹데이터 기반 한국어 말뭉치 데이터/Validation/원천데이터/VS1/정치/' + '**/*.json',
'AIHUB_웹데이터 기반 한국어 말뭉치 데이터/Validation/원천데이터/VS1/지역/' + '**/*.json',
'AIHUB_웹데이터 기반 한국어 말뭉치 데이터/Validation/원천데이터/VS1/취미/' + '**/*.json',]

train_file_name_01 = glob(path_list[0], recursive = True)
train_file_name_02 = glob(path_list[1], recursive = True)
train_file_name_03 = glob(path_list[2], recursive = True)
train_file_name_04 = glob(path_list[3], recursive = True)
train_file_name_05 = glob(path_list[4], recursive = True)
train_file_name_06 = glob(path_list[5], recursive = True)
train_file_name_07 = glob(path_list[6], recursive = True)
train_file_name_08 = glob(path_list[7], recursive = True)
train_file_name_09 = glob(path_list[8], recursive = True)
train_file_name_10 = glob(path_list[9], recursive = True)
train_file_name_11 = glob(path_list[10], recursive = True)
train_file_name_12 = glob(path_list[11], recursive = True)
train_file_name_13 = glob(path_list[12], recursive = True)
train_file_name_14 = glob(path_list[13], recursive = True)
train_file_name_15 = glob(path_list[14], recursive = True)
train_file_name_16 = glob(path_list[15], recursive = True)
train_file_name_17 = glob(path_list[16], recursive = True)

valid_file_name_01 = glob(path_list[17], recursive = True)
valid_file_name_02 = glob(path_list[18], recursive = True)
valid_file_name_03 = glob(path_list[19], recursive = True)
valid_file_name_04 = glob(path_list[20], recursive = True)
valid_file_name_05 = glob(path_list[21], recursive = True)
valid_file_name_06 = glob(path_list[22], recursive = True)
valid_file_name_07 = glob(path_list[23], recursive = True)
valid_file_name_08 = glob(path_list[24], recursive = True)
valid_file_name_09 = glob(path_list[25], recursive = True)
valid_file_name_10 = glob(path_list[26], recursive = True)
valid_file_name_11 = glob(path_list[27], recursive = True)
valid_file_name_12 = glob(path_list[28], recursive = True)
valid_file_name_13 = glob(path_list[29], recursive = True)
valid_file_name_14 = glob(path_list[30], recursive = True)
valid_file_name_15 = glob(path_list[31], recursive = True)
valid_file_name_16 = glob(path_list[32], recursive = True)
valid_file_name_17 = glob(path_list[33], recursive = True)

In [508]:
print(train_file_name_13[0])

with open(train_file_name_13[0], 'r', encoding='utf-8') as one_json_file:
    one_json_sample = json.load(one_json_file)

source_list = list(pd.DataFrame(one_json_sample['SJML']['text'])['content'])

AIHUB_웹데이터 기반 한국어 말뭉치 데이터/Training/원천데이터/TS1/여행레저\BWTL217000058479.json


In [509]:
source_list[6]

'. 행정자치부가 자전거 주간을 맞아 23일 머니위크가 주최한 청계릴레이사이클링(청계천 가족자전거 운동회)를 공식후원하고 현장에서 자전거 안전 캠페인을 실시했다.. . 자전거 정책을 맡은 행정자치부 주민생활환경과는 청계천 한빛광장에 마련된 무대에서 운동회 참가자와 주말 청계천 나들이객을 대상으로 자전거 안전운전요령 등 홍보책자, 전조등 등 기념품 등을 배부하고 안전한 자전거타기 홍보 캠페인을 실시했다.. . 이날 행사에서 허만영 행정자치부 주민생활환경과장은 “최근 자전거가 1000만대 수준으로 크게 늘면서 자전거사고도 매년 10%씩 증가하고 있다”면서 “이제는 안전하게 자전거를 타는 것이 무엇보다도 중요하다”고 강조하며 행사에 참여한 시민들에게 자전거 안전문화 확산에 적극 동참해줄 것을 당부했다.. . 한편 행정자치부는 2016년도 자전거도로정비 사업비 20억원을 지자체에 지원해 생활형 자전거이용이 많은 도심지 자전거·보행자겸용도로정비 개선사업을 추진해 안전하고 편리한 자전거 (이름)경을 (이름)다.. . 또한 자전거사고 취약계층인 어린이와 어르신들의 자전거 안전사고 예방을 위해 자전거 민간단체와 협업을 통해 자전거 안전교실을 운영(5월~11월)한다.'

In [510]:
preprocessing_sentences = formal_preprocessing_text(source_list[6])
for preprocessing_sentence in preprocessing_sentences:
    print(preprocessing_sentence, end="\n\n")

자전거 정책을 맡은 행정자치부 주민생활환경과는 청계천 한빛광장에 마련된 무대에서 운동회 참가자와 주말 청계천 나들이객을 대상으로 자전거 안전운전요령 등 홍보책자, 전조등 등 기념품 등을 배부하고 안전한 자전거타기 홍보 캠페인을 실시했다.. .

이날 행사에서 허만영 행정자치부 주민생활환경과장은 최근 자전거가 1000만대 수준으로 크게 늘면서 자전거사고도 매년 10 씩 증가하고 있다 면서 이제는 안전하게 자전거를 타는 것이 무엇보다도 중요하다 고 강조하며 행사에 참여한 시민들에게 자전거 안전문화 확산에 적극 동참해줄 것을 당부했다.. .

한편 행정자치부는 2016년도 자전거도로정비 사업비 20억원을 지자체에 지원해 생활형 자전거이용이 많은 도심지 자전거 보행자겸용도로정비 개선사업을 추진해 안전하고 편리한 자전거 (이름)경을 (이름)다.. .

또한 자전거사고 취약계층인 어린이와 어르신들의 자전거 안전사고 예방을 위해 자전거 민간단체와 협업을 통해 자전거 안전교실을 운영(5월 11월)한다.



### AIHUB 산업정보 연계 주요국 특허 영-한 데이터

[Source](https://aihub.or.kr/aihubdata/data/view.do?currMenu=&topMenu=&aihubDataSe=realm&dataSetSn=563)

In [511]:
path_list = ['AIHUB_산업정보 연계 주요국 특허 영-한 데이터/Training/'+ '/**/*.json', 
             'AIHUB_산업정보 연계 주요국 특허 영-한 데이터/Validation/'+ '/**/*.json']
train_file_name, valid_file_name = train_valid_json_file_name_list(path_list)

In [512]:
source_file = train_file_name[130]
print(source_file)

with open(source_file, 'r', encoding='utf-8') as one_json_file:
    one_json_sample = json.load(one_json_file)
    
try: 
    source_list = list(pd.DataFrame(one_json_sample['labeled_data'])['astrt_cont_kor'])

except:
    pass   

AIHUB_산업정보 연계 주요국 특허 영-한 데이터/Training\라벨링데이터_0831_add\TL_1\금속 절삭기계 제조업.json


In [513]:
source_list[79]

'종래 제품의 단점 하나 또는 하나 이상을 해소하는 왕복 블레이드를 제공하는 것이다. 반복 패턴 16이 3개의 절삭치아 18을 포함하고 절삭 방향 A에 관해 선행치아 20이 선행하고 선행치아 20에 2개의 후속치아 22 및 24가 계속된다. 선행치아 20은 선단부 26, 제 1떠올려 모서리α를 형성하는 떠올려 면 28, 주도망면 30, 날끝각 32를 포함한다. 제1 후속치아 22는 선단부 34, 제 2떠올려 모서리β1을 형성하는 떠올려 면 36, 주도망면 38, 날끝각 40을 포함한다. 제2 후속치아 24는 선단부 42, 다른 제 2떠올려 모서리β2를 형성하는 떠올려 면 44, 주도망면 46, 날끝각 48을 포함한다. 후속치아 22 및 24의 프로파일은 동일하고 선단부 34 및 42는 동일 높이이며 제 2떠올려 모서리β1과β2와는 동일 각도를 형성하고 날끝각 40 및 48은 동일 각도를 형성한다.'

In [515]:
preprocessing_sentences = legal_preprocessing_text(source_list[79])
for preprocessing_sentence in preprocessing_sentences:
    print(preprocessing_sentence, end="\n\n")

종래 제품의 단점 하나 또는 하나 이상을 해소하는 왕복 블레이드를 제공하는 것이다.

반복 패턴 16이 3개의 절삭치아 18을 포함하고 절삭 방향 A에 관해 선행치아 20이 선행하고 선행치아 20에 2개의 후속치아 22 및 24가 계속된다.

선행치아 20은 선단부 26, 제 1떠올려 모서리 를 형성하는 떠올려 면 28, 주도망면 30, 날끝각 32를 포함한다.

제1 후속치아 22는 선단부 34, 제 2떠올려 모서리 1을 형성하는 떠올려 면 36, 주도망면 38, 날끝각 40을 포함한다.

제2 후속치아 24는 선단부 42, 다른 제 2떠올려 모서리 2를 형성하는 떠올려 면 44, 주도망면 46, 날끝각 48을 포함한다.

후속치아 22 및 24의 프로파일은 동일하고 선단부 34 및 42는 동일 높이이며 제 2떠올려 모서리 1과 2와는 동일 각도를 형성하고 날끝각 40 및 48은 동일 각도를 형성한다.



### AIHUB 논문자료 요약

[Source](https://aihub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100&aihubDataSe=realm&dataSetSn=90)

In [516]:
path_list = ['AIHUB_논문자료 요약/Training/'+ '/**/*.json', 
             'AIHUB_논문자료 요약/Validation/'+ '/**/*.json']
train_file_name, valid_file_name = train_valid_json_file_name_list(path_list)

In [517]:
source_file = train_file_name[0]
print(source_file)

with open(source_file, 'r', encoding='utf-8') as one_json_file:
    one_json_sample = json.load(one_json_file)

source_df = pd.DataFrame(one_json_sample['data'])
    
if '논문/논문요약' or '특허섹션' in source_file:
    try:
        summary_entire_source_dict = dict(source_df['summary_entire'].explode())
        summary_entire_source_json = pd.json_normalize(summary_entire_source_dict)
        summary_entire_source_list = list(summary_entire_source_json.filter(regex='orginal_text').values[0])

        summary_section_source_dict = dict(source_df['summary_section'].explode())
        summary_section_source_json = pd.json_normalize(summary_section_source_dict)
        summary_section_source_list = list(summary_section_source_json.filter(regex='orginal_text').values[0])

        source_list = summary_entire_source_list + summary_section_source_list
   
    except:
        summary_section_source_dict = dict(source_df['summary_section'].explode())
        summary_section_source_json = pd.json_normalize(summary_section_source_dict)
        source_list = list(summary_section_source_json.filter(regex='orginal_text').values[0])
        
else:
    summary_section_source_dict = dict(source_df['summary_section'].explode())
    summary_section_source_json = pd.json_normalize(summary_section_source_dict)
    source_list = list(summary_section_source_json.filter(regex='orginal_text').values[0])

AIHUB_논문자료 요약/Training\training_특허섹션만\9-N-0.json


In [518]:
source_list[980]

'종래, 시접을 일정하게 유지하기 위해, 상하로 겹쳐진 2장의 피봉제물(천)의 단부 가장자리를 맞추면서 봉합하는 재봉기는, 2장의 천을 천 이송방향으로 이송하는 상하이송수단이 설치되는 동시에, 상기 상하이송수단에 대해 천 이송방향 상류측에 배치되고, 천의 천 이송방향을 따르는 일단(一端) 가장자리를 일치시키도록 천을 천 이송방향과의 교차방향으로 이동시키는 천 이동장치가 설치되어 있다.상기 천 이동장치는, 바늘판 상면에서 천 이송방향과 교차하는 방향으로 각각의 천을 이동시키도록, 천(C2,C1)에 각각 따로 접하며 천 이송방향을 따르는 축선 중심으로 회전하는 롤러를 갖는 상하 매니퓰레이터(manipulator)와, 2장의 천의 각 일단 가장자리를 광학적으로 검출하는 검지수단과, 상기 검지수단의 출력에 의해 상기 상하 매니퓰레이터의 각 롤러를 각각 따로 회전시키는 펄스모터를 구비하고 있다.그런데, 상기 봉합하기 위해 사용되는 피봉제물은, 섬유에 의한 옷감(生地)이 많이 사용되며, 그리고 옷감을 천 패턴(布型)에 맞춰 절단하여 형상을 조정하고 있다.  이러한 옷감은, 절단된 천 단부에서 옷감의 섬유가 풀려, 천 단부로부터 섬유실이 튀어 나온 상태가 자주 발생한다.  이 때문에, 천 단부를 광학적으로 검출하는 검지수단이 튀어나온 섬유실(이후, ‘보풀’이라 함)을 천 단부로 오(誤)인식하여, 피봉제물을 옷감 안쪽으로 이동시키므로, 천(C2,C1)의 단부 가장자리가 가지런하지 않게 되거나, 바늘이 옷감으로부터 벗어난 위치로 하강하여 바늘땀을 매듭지을 수 없게 될 우려가 있었다.따라서, 예컨대, 특허문헌 1에 기재된 재봉기에서는, 천 단부를 검출하는 수광소자로 이루어진 천 단부센서에 덧붙여, 상기 천 단부센서의 외측(천 단부를 경계로 하는 경우에서의 천의 반대쪽)에 수광소자로 이루어진 보풀검출센서를 설치한다는 구성을 채용하고 있다.이러한 재봉기에서는, 천 단부의 위치는 천 단부센서의 수광량의 변화에 따라 검출하고, 보풀에 대해서는 천 단부센서와 보풀검출센서의 양쪽에서 검출

In [519]:
preprocessing_sentences = formal_preprocessing_text(source_list[980])
for preprocessing_sentence in preprocessing_sentences:
    print(preprocessing_sentence, end="\n\n")

종래, 시접을 일정하게 유지하기 위해, 상하로 겹쳐진 2장의 피봉제물(천)의 단부 가장자리를 맞추면서 봉합하는 재봉기는, 2장의 천을 천 이송방향으로 이송하는 상하이송수단이 설치되는 동시에, 상기 상하이송수단에 대해 천 이송방향 상류측에 배치되고, 천의 천 이송방향을 따르는 일단(一端) 가장자리를 일치시키도록 천을 천 이송방향과의 교차방향으로 이동시키는 천 이동장치가 설치되어 있다.

상기 천 이동장치는, 바늘판 상면에서 천 이송방향과 교차하는 방향으로 각각의 천을 이동시키도록, 천(C2,C1)에 각각 따로 접하며 천 이송방향을 따르는 축선 중심으로 회전하는 롤러를 갖는 상하 매니퓰레이터(manipulator)와, 2장의 천의 각 일단 가장자리를 광학적으로 검출하는 검지수단과, 상기 검지수단의 출력에 의해 상기 상하 매니퓰레이터의 각 롤러를 각각 따로 회전시키는 펄스모터를 구비하고 있다.

그런데, 상기 봉합하기 위해 사용되는 피봉제물은, 섬유에 의한 옷감(生地)이 많이 사용되며, 그리고 옷감을 천 패턴(布型)에 맞춰 절단하여 형상을 조정하고 있다.

이러한 옷감은, 절단된 천 단부에서 옷감의 섬유가 풀려, 천 단부로부터 섬유실이 튀어 나온 상태가 자주 발생한다.

이 때문에, 천 단부를 광학적으로 검출하는 검지수단이 튀어나온 섬유실(이후, 보풀 이라 함)을 천 단부로 오(誤)인식하여, 피봉제물을 옷감 안쪽으로 이동시키므로, 천(C2,C1)의 단부 가장자리가 가지런하지 않게 되거나, 바늘이 옷감으로부터 벗어난 위치로 하강하여 바늘땀을 매듭지을 수 없게 될 우려가 있었다.

따라서, 예컨대, 특허문헌 1에 기재된 재봉기에서는, 천 단부를 검출하는 수광소자로 이루어진 천 단부센서에 덧붙여, 상기 천 단부센서의 외측(천 단부를 경계로 하는 경우에서의 천의 반대쪽)에 수광소자로 이루어진 보풀검출센서를 설치한다는 구성을 채용하고 있다.

이러한 재봉기에서는, 천 단부의 위치는 천 단부센서의 수광량의 변화에 따라 검출하고, 보풀에 대해서는 천 단부센서와 보풀검출센서의 양

In [520]:
source_file = train_file_name[18]
print(source_file)

with open(source_file, 'r', encoding='utf-8') as one_json_file:
    one_json_sample = json.load(one_json_file)
    
if '특허_z' in source_file:
    source_df = pd.DataFrame(one_json_sample['data'])
    source_list = list(source_df[source_df['attr'] == 'abs']['text'])


elif '특허_0' in source_file or '특허_1' in source_file:
    source_df = pd.DataFrame(one_json_sample['data'])
    source_dict = dict(source_df[source_df['attr'] == '초록']['sentence'].explode())
    source_json = pd.json_normalize(source_dict)
    source_list = list(source_json.filter(regex='text').values[0])            


elif '논문/논문요약' or '특허섹션' in source_file:
    
    source_df = pd.DataFrame(one_json_sample['data'])
    
    try:
        summary_entire_source_dict = dict(source_df['summary_entire'].explode())
        summary_entire_source_json = pd.json_normalize(summary_entire_source_dict)
        summary_entire_source_list = list(summary_entire_source_json.filter(regex='orginal_text').values[0])

        summary_section_source_dict = dict(source_df['summary_section'].explode())
        summary_section_source_json = pd.json_normalize(summary_section_source_dict)
        summary_section_source_list = list(summary_section_source_json.filter(regex='orginal_text').values[0])  

        source_list = summary_entire_source_list + summary_section_source_list   

    except:
        summary_section_source_dict = dict(source_df['summary_section'].explode())
        summary_section_source_json = pd.json_normalize(summary_section_source_dict)
        source_list = list(summary_section_source_json.filter(regex='orginal_text').values[0])

else:
    source_df = pd.DataFrame(one_json_sample['data'])
    summary_section_source_dict = dict(source_df['summary_section'].explode())
    summary_section_source_json = pd.json_normalize(summary_section_source_dict)
    source_list = list(summary_section_source_json.filter(regex='orginal_text').values[0])    

AIHUB_논문자료 요약/Training\training_논문\논문요약_0225_7_0.json


In [521]:
source_list[980]

'본 논문에서는 환경지도 제작 활동이 학습자의 환경 감수성 발달 및 환경에 대한 인식 변화에 유의미한 영향을 줄 수 있는지 탐색해보는 것을 그 목적으로 한다. 본 연구의 대상은 서울C초등학교 5학년 학생 21명(남 11명, 여 10명)이었다. 학생들은 우리 동네의 환경문제에 대해 논의하고, 우리 동네 지도를 그렸다. 또한 논의한 환경문제(미세 먼지, 소음)를 측정하기 위해 야외 답사를 실시하였고, 야외 답사의 결과를 종합하여 우리 동네 환경문제의 심각성을 도출하였다. 마지막으로 학생들은 구청에 청원서를 작성하는 활동을 통해 우리 지역의 문제를 스스로 해결해 보는 활동에 참여하였다. 10차시에 걸친 환경지도 만들기 활동을 통해 학생들의 환경 감수성이 유의미하게 발달하였는지 알아보기 위해 이재붕·이두곤(2006)이 개발한 환경 감수성 검사 도구를 활용하였다. 이를 통한 본 연구의 결과는 다음과 같다. 첫째, 학생들의 환경에 대한 인식에 있어 유의미한 인식 변화가 있음을 알 수 있었다. 둘째, 환경지도의 제작을 통해 학생들의 환경 감수성에도 유의미한 인식 변화가 나타났음을 알 수 있었다. 셋째, 학생들의 사후 인터뷰를 통해 환경지도 제작 과정에서 우리 동네 환경에 대한 관심이 생겼으며, 청원서를 작성하는 경험을 통해 민주시민으로서 책임감을 느낄 수 있었음을 알 수 있었다. 따라서 환경지도 그리기 활동은 학생들이 환경 감수성을 가진 생태 시민으로서 책임감을 가지고 일상의 문제에 능동적으로 참여할 수 있는 기회를 제공해 주었다는 측면에서 그 의의를 가질 수 있다.'

In [522]:
preprocessing_sentences = formal_preprocessing_text(source_list[980])
for preprocessing_sentence in preprocessing_sentences:
    print(preprocessing_sentence, end="\n\n")

본 논문에서는 환경지도 제작 활동이 학습자의 환경 감수성 발달 및 환경에 대한 인식 변화에 유의미한 영향을 줄 수 있는지 탐색해보는 것을 그 목적으로 한다.

본 연구의 대상은 서울C초등학교 5학년 학생 21명(남 11명, 여 10명)이었다.

학생들은 우리 동네의 환경문제에 대해 논의하고, 우리 동네 지도를 그렸다.

또한 논의한 환경문제(미세 먼지, 소음)를 측정하기 위해 야외 답사를 실시하였고, 야외 답사의 결과를 종합하여 우리 동네 환경문제의 심각성을 도출하였다.

마지막으로 학생들은 구청에 청원서를 작성하는 활동을 통해 우리 지역의 문제를 스스로 해결해 보는 활동에 참여하였다.

10차시에 걸친 환경지도 만들기 활동을 통해 학생들의 환경 감수성이 유의미하게 발달하였는지 알아보기 위해 이재붕 이두곤(2006)이 개발한 환경 감수성 검사 도구를 활용하였다.

이를 통한 본 연구의 결과는 다음과 같다.

첫째, 학생들의 환경에 대한 인식에 있어 유의미한 인식 변화가 있음을 알 수 있었다.

둘째, 환경지도의 제작을 통해 학생들의 환경 감수성에도 유의미한 인식 변화가 나타났음을 알 수 있었다.

셋째, 학생들의 사후 인터뷰를 통해 환경지도 제작 과정에서 우리 동네 환경에 대한 관심이 생겼으며, 청원서를 작성하는 경험을 통해 민주시민으로서 책임감을 느낄 수 있었음을 알 수 있었다.

따라서 환경지도 그리기 활동은 학생들이 환경 감수성을 가진 생태 시민으로서 책임감을 가지고 일상의 문제에 능동적으로 참여할 수 있는 기회를 제공해 주었다는 측면에서 그 의의를 가질 수 있다.



In [523]:
source_file = train_file_name[48]
print(source_file)

with open(source_file, 'r', encoding='utf-8') as one_json_file:
    one_json_sample = json.load(one_json_file)
    
if '특허_z' in source_file:
    source_df = pd.DataFrame(one_json_sample['data'])
    source_list = list(source_df[source_df['attr'] == 'abs']['text'])


elif '특허_0' in source_file or '특허_1' in source_file:
    source_df = pd.DataFrame(one_json_sample['data'])
    source_dict = dict(source_df[source_df['attr'] == '초록']['sentence'].explode())
    source_json = pd.json_normalize(source_dict)
    source_list = list(source_json.filter(regex='text').values[0])            


elif '논문/논문요약' or '특허섹션' in source_file:
    
    source_df = pd.DataFrame(one_json_sample['data'])
    
    try:
        summary_entire_source_dict = dict(source_df['summary_entire'].explode())
        summary_entire_source_json = pd.json_normalize(summary_entire_source_dict)
        summary_entire_source_list = list(summary_entire_source_json.filter(regex='orginal_text').values[0])

        summary_section_source_dict = dict(source_df['summary_section'].explode())
        summary_section_source_json = pd.json_normalize(summary_section_source_dict)
        summary_section_source_list = list(summary_section_source_json.filter(regex='orginal_text').values[0])  

        source_list = summary_entire_source_list + summary_section_source_list   

    except:
        summary_section_source_dict = dict(source_df['summary_section'].explode())
        summary_section_source_json = pd.json_normalize(summary_section_source_dict)
        source_list = list(summary_section_source_json.filter(regex='orginal_text').values[0])

else:
    source_df = pd.DataFrame(one_json_sample['data'])
    summary_section_source_dict = dict(source_df['summary_section'].explode())
    summary_section_source_json = pd.json_normalize(summary_section_source_dict)
    source_list = list(summary_section_source_json.filter(regex='orginal_text').values[0])    

AIHUB_논문자료 요약/Training\training_특허전체\특허전체_9-Y-29.json


In [526]:
print(source_list[10])


[ 발명의 명칭 ]
담배 필터에 내장되는 보조 필터auxiliary filter for built in tobacco filter
[ 기술분야 ]
본 발명은 담배 필터에 내장되는 보조 필터에 관한 것으로, 더욱 상세하게는 흡연시 담배 연기에 함유된 니코틴, 타르 등의 유해 성분을 2차적으로 필터링함으로써 니코틴 및 타르 등 유해 성분의 흡입을 최소화할 수 있도록 할 뿐만 아니라 담배 연기에 함유된 수분을 2차적으로 필터링함으로써 담배 연기의 수분 과다 함유로 인한 담배 맛 저하를 방지할 수 있도록 하는 담배 필터에 내장되는 보조 필터에 관한 것이다.
[ 배경기술 ]
담배 연기에는 니코틴, 타르 등을 포함하는 각종의 유해 성분이 상당량 포함된다.따라서, 흡연자에게서는 니코틴, 타르 등을 포함하는 각종 유해 성분으로 인한 질환 발생률이 상당하다.이러한 이유로 최근 사회적으로 금연 분위기가 조성되고 있다.그러나, 흡연자의 상당수는 중독성 및 습관성으로 인하여 금연에 이르지 못하고 있어, 흡연으로 인한 질환 발생은 여전히 이어지고 있다.한편, 담배 제조업체에서는 흡연시 니코틴, 타르 등을 포함하는 각종 유해 성분으로 인한 피해를 최소화하기 위하여 담배 로드 말단에 대한민국 등록특허공보 제10-1340673호(공고일 : 2013. 12. 12.) 등에 개시된 바와 같은 담배 필터를 적용하고 있다.담배 필터는, 통상 섬유질 구간을 포함하는바, 흡연시 담배 연기에 함유된 니코틴, 타르 등의 유해 성분이 섬유질 구간을 거치는 과정에서 필터링될 수 있어 니코틴, 타르 등의 유해 성분 흡입이 최소화된다.그러나, 종래 담배 필터는 필터링 과정에서 섬유질 구간의 공극 등이 점차로 막히므로 니코틴, 타르 등의 유해 성분 필터링 효과가 저하되는 문제가 있었다.또한, 종래 담배 필터는, 흡연시 담배 연기에 함유된 수분의 필터링 효과가 기대에 미치지 못하는바, 담배 연기의 과도한 수분 함유에 의해 매운맛 등이 강해져 담배 고유의 맛이 저하되는 문제가 있었다.상기의 이유로 해당분야에서는 

In [527]:
preprocessing_sentences = formal_preprocessing_text(source_list[10])
for preprocessing_sentence in preprocessing_sentences:
    print(preprocessing_sentence, end="\n\n")

본 발명은 담배 필터에 내장되는 보조 필터에 관한 것으로, 더욱 상세하게는 흡연시 담배 연기에 함유된 니코틴, 타르 등의 유해 성분을 2차적으로 필터링함으로써 니코틴 및 타르 등 유해 성분의 흡입을 최소화할 수 있도록 할 뿐만 아니라 담배 연기에 함유된 수분을 2차적으로 필터링함으로써 담배 연기의 수분 과다 함유로 인한 담배 맛 저하를 방지할 수 있도록 하는 담배 필터에 내장되는 보조 필터에 관한 것이다.

담배 연기에는 니코틴, 타르 등을 포함하는 각종의 유해 성분이 상당량 포함된다.

따라서, 흡연자에게서는 니코틴, 타르 등을 포함하는 각종 유해 성분으로 인한 질환 발생률이 상당하다.

이러한 이유로 최근 사회적으로 금연 분위기가 조성되고 있다.

그러나, 흡연자의 상당수는 중독성 및 습관성으로 인하여 금연에 이르지 못하고 있어, 흡연으로 인한 질환 발생은 여전히 이어지고 있다.

한편, 담배 제조업체에서는 흡연시 니코틴, 타르 등을 포함하는 각종 유해 성분으로 인한 피해를 최소화하기 위하여 담배 로드 말단에 대한민국 등록특허공보 제10-1340673호(공고일 2013. 12. 12.) 등에 개시된 바와 같은 담배 필터를 적용하고 있다.

담배 필터는, 통상 섬유질 구간을 포함하는바, 흡연시 담배 연기에 함유된 니코틴, 타르 등의 유해 성분이 섬유질 구간을 거치는 과정에서 필터링될 수 있어 니코틴, 타르 등의 유해 성분 흡입이 최소화된다.

그러나, 종래 담배 필터는 필터링 과정에서 섬유질 구간의 공극 등이 점차로 막히므로 니코틴, 타르 등의 유해 성분 필터링 효과가 저하되는 문제가 있었다.

또한, 종래 담배 필터는, 흡연시 담배 연기에 함유된 수분의 필터링 효과가 기대에 미치지 못하는바, 담배 연기의 과도한 수분 함유에 의해 매운맛 등이 강해져 담배 고유의 맛이 저하되는 문제가 있었다.

상기의 이유로 해당분야에서는 흡연시 담배 연기에 함유된 니코틴, 타르 등 유해 성분의 필터링 효과를 높일 수 있도록 함으로써 니코틴, 타르 등 유해 성분의 흡입을 최소

### AIHUB 요약문 및 레포트 생성 데이터

[Source](https://www.aihub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100&aihubDataSe=realm&dataSetSn=582)

In [528]:
path_list = ['AIHUB_요약문 및 레포트 생성 데이터/Training/원천데이터/TS1/01.news_r/' + '**/*.json',
'AIHUB_요약문 및 레포트 생성 데이터/Training/원천데이터/TS1/02.briefing/' + '**/*.json',
'AIHUB_요약문 및 레포트 생성 데이터/Training/원천데이터/TS1/03.his_cul/' + '**/*.json',
'AIHUB_요약문 및 레포트 생성 데이터/Training/원천데이터/TS1/04.paper/' + '**/*.json',
'AIHUB_요약문 및 레포트 생성 데이터/Training/원천데이터/TS1/05.minute/' + '**/*.json',
'AIHUB_요약문 및 레포트 생성 데이터/Training/원천데이터/TS1/06.edit/' + '**/*.json',
'AIHUB_요약문 및 레포트 생성 데이터/Training/원천데이터/TS1/07.public/' + '**/*.json',
'AIHUB_요약문 및 레포트 생성 데이터/Training/원천데이터/TS1/08.speech/' + '**/*.json',
'AIHUB_요약문 및 레포트 생성 데이터/Training/원천데이터/TS1/09.literature/' + '**/*.json',
'AIHUB_요약문 및 레포트 생성 데이터/Training/원천데이터/TS1/10.narration/' + '**/*.json',
'AIHUB_요약문 및 레포트 생성 데이터/Validation/원천데이터/VS1/01.news_r/' + '**/*.json',
'AIHUB_요약문 및 레포트 생성 데이터/Validation/원천데이터/VS1/02.briefing/' + '**/*.json',
'AIHUB_요약문 및 레포트 생성 데이터/Validation/원천데이터/VS1/03.his_cul/' + '**/*.json',
'AIHUB_요약문 및 레포트 생성 데이터/Validation/원천데이터/VS1/04.paper/' + '**/*.json',
'AIHUB_요약문 및 레포트 생성 데이터/Validation/원천데이터/VS1/05.minute/' + '**/*.json',
'AIHUB_요약문 및 레포트 생성 데이터/Validation/원천데이터/VS1/06.edit/' + '**/*.json',
'AIHUB_요약문 및 레포트 생성 데이터/Validation/원천데이터/VS1/07.public/' + '**/*.json',             
'AIHUB_요약문 및 레포트 생성 데이터/Validation/원천데이터/VS1/08.speech/' + '**/*.json',
'AIHUB_요약문 및 레포트 생성 데이터/Validation/원천데이터/VS1/09.literature/' + '**/*.json',
'AIHUB_요약문 및 레포트 생성 데이터/Validation/원천데이터/VS1/10.narration/' + '**/*.json']

train_file_name_01 = glob(path_list[0], recursive = True)
train_file_name_02 = glob(path_list[1], recursive = True)
train_file_name_03 = glob(path_list[2], recursive = True)
train_file_name_04 = glob(path_list[3], recursive = True)
train_file_name_05 = glob(path_list[4], recursive = True)
train_file_name_06 = glob(path_list[5], recursive = True)
train_file_name_07 = glob(path_list[6], recursive = True)
train_file_name_08 = glob(path_list[7], recursive = True)
train_file_name_09 = glob(path_list[8], recursive = True)
train_file_name_10 = glob(path_list[9], recursive = True)
valid_file_name_01 = glob(path_list[10], recursive = True)
valid_file_name_02 = glob(path_list[11], recursive = True)
valid_file_name_03 = glob(path_list[12], recursive = True)
valid_file_name_04 = glob(path_list[13], recursive = True)
valid_file_name_05 = glob(path_list[14], recursive = True)
valid_file_name_06 = glob(path_list[15], recursive = True)
valid_file_name_07 = glob(path_list[16], recursive = True)
valid_file_name_08 = glob(path_list[17], recursive = True)
valid_file_name_09 = glob(path_list[18], recursive = True)
valid_file_name_10 = glob(path_list[19], recursive = True)  

In [529]:
source_file = train_file_name_01[799]
print(source_file)

with open(source_file, 'r', encoding='utf-8') as one_json_file:
    one_json_sample = json.load(one_json_file)

source = one_json_sample['Meta(Refine)']['passage']

AIHUB_요약문 및 레포트 생성 데이터/Training/원천데이터/TS1/01.news_r\20per\REPORT-news_r-02275-00002.json


In [530]:
print(source)

 하지만 크런치베이스에는 CB인사이츠에 포함된 바이오기업 에이프로젠 대신 이커머스 기업 티몬이 올라있다.
  중소벤처기업부 등 국내 정부기관에서는 CB인사이츠의 자료를 활용한다.
  정유신 서강대 경영대 교수는 "조사기관들은 보통 투자액을 통해 기업가치를 평가하는데, 투자액의 인정 기준이 조사기관마다 다를 수 있다"며 "글로벌 기준이 되는 미국의 조사결과를 더 신뢰하는 경향이 있다"고 설명했다.
   ③국가 지원받는 중국, 데카콘만 6곳  CB인사이츠에서는 유니콘 기업의 분야를 16개로 나눴다.
  핀테크 스타트업이 60곳으로 가장 많다.
  이커머스(54곳)-인터넷서비스(54곳)-인공지능(46곳) 순이다.
  미국 유니콘은 인터넷 서비스(43곳)-핀테크(30곳), 중국은 이커머스(21곳)-인공지능(12곳) 순이다.
  한국은 쿠팡, 위메프, 무신사 등 이커머스 기업이 3곳으로 가장 많다.
   기업가치가 유니콘의 10배 이상인 기업을 데카콘이라 부른다.
  기업가치 100억 달러(11조 7000억원) 이상인 유니콘, 즉 데카콘은 총 23곳이다.
  미국이 11곳이고, 중국이 6곳이다.
  기업가치가 가장 높은 기업 1, 2위는 모두 중국 기업이다.
  정 교수는 "중국에서는 기술 기업을 전폭적으로 지원해주고 있다"고 말했다.
  1위는 동영상 플랫폼 '틱톡'을 서비스하는 바이트댄스(750억 달러), 2위는 '중국의 우버' 디디추싱(560억 달러)이다.
  아시아 데카콘은 중국(6곳)을 포함해 인도 2곳, 인도네시아·싱가포르 1곳 등 10곳이다.
  한국에선 쿠팡의 기업가치가 90억 달러(약 10조 500억원)로 가장 높다.
   ④한국 10곳 중 5곳 적자   유니콘 기업이 잇달아 등장하자 정부에서는 적극적으로 육성 정책을 내놓고 있다.
  하지만 전문가들은 유니콘에 대한 맹목적인 신뢰를 경계해야 한다고 말한다.
  이병태 교수는 "유니콘 타이틀을 갖게 되면 언론의 주목을 받게 되고 추가 투자를 받기 유리해진다"며 "기업 존립이 위태로운 기업이 유니콘으로 대

In [531]:
preprocessing_sentences = formal_preprocessing_text(source)
for preprocessing_sentence in preprocessing_sentences:
    print(preprocessing_sentence, end="\n\n")

하지만 크런치베이스에는 CB인사이츠에 포함된 바이오기업 에이프로젠 대신 이커머스 기업 티몬이 올라있다.

중소벤처기업부 등 국내 정부기관에서는 CB인사이츠의 자료를 활용한다.

정유신 서강대 경영대 교수는 조사기관들은 보통 투자액을 통해 기업가치를 평가하는데, 투자액의 인정 기준이 조사기관마다 다를 수 있다 며 글로벌 기준이 되는 미국의 조사결과를 더 신뢰하는 경향이 있다 고 설명했다.

미국 유니콘은 인터넷 서비스(43곳)-핀테크(30곳), 중국은 이커머스(21곳)-인공지능(12곳) 순이다.

한국은 쿠팡, 위메프, 무신사 등 이커머스 기업이 3곳으로 가장 많다.

기업가치가 유니콘의 10배 이상인 기업을 데카콘이라 부른다.

기업가치 100억 달러(11조 7000억원) 이상인 유니콘, 즉 데카콘은 총 23곳이다.

기업가치가 가장 높은 기업 1, 2위는 모두 중국 기업이다.

정 교수는 중국에서는 기술 기업을 전폭적으로 지원해주고 있다 고 말했다.

1위는 동영상 플랫폼 틱톡 을 서비스하는 바이트댄스(750억 달러), 2위는 중국의 우버 디디추싱(560억 달러)이다.

아시아 데카콘은 중국(6곳)을 포함해 인도 2곳, 인도네시아 싱가포르 1곳 등 10곳이다.

한국에선 쿠팡의 기업가치가 90억 달러(약 10조 500억원)로 가장 높다.

하지만 전문가들은 유니콘에 대한 맹목적인 신뢰를 경계해야 한다고 말한다.

이병태 교수는 유니콘 타이틀을 갖게 되면 언론의 주목을 받게 되고 추가 투자를 받기 유리해진다 며 기업 존립이 위태로운 기업이 유니콘으로 대접받는 경우도 있는데, 거품이 끼어 있는 것 이라고 설명했다.

실제로 한국의 유니콘 10곳 가운데 5곳이 적자(2018년 기준 영업손실)다.

또 전 세계적으로 유니콘의 최소 기준인 기업가치 1억 달러 기업이 전체의 33 (146개)나 된다.

익명을 요청한 벤처캐피탈 관계자는 정부는 규제개혁을 통해 새로운 실험이 일어나게 하고, 기초과학에 투자해 인적자산 확보에 나서야 한다.



In [532]:
source_file = train_file_name_09[932]
print(source_file)

with open(source_file, 'r', encoding='utf-8') as one_json_file:
    one_json_sample = json.load(one_json_file)

source = one_json_sample['Meta(Refine)']['passage']

AIHUB_요약문 및 레포트 생성 데이터/Training/원천데이터/TS1/09.literature\20per\REPORT-literature-08132-08132.json


In [533]:
print(source)

 그러나 자연스런 그들의 행위가 반면에 의외로 큰 희생을 요구했으니 그것은 단체에 끼치게 된 불리보다도 참으로 크리이긴과 삑톨 두 사람에게 던지게 된 불행이다. 삑톨이 조바심을 하고 안달을 하면서 두 사람의 종적을 찾으러 휘돌아치는 꼴에는 단의 책임자로서의 심정보다도 마리이에게 대한 실망과 초조가 드러나 보이는 듯하다. 며칠 전 호텔 병실에서 그의 아내 그라아샤가 마리이를 냉큼 내쫓아 달라고 고함을 쳤던 것이 그럴 필요조차 없게 제물에 해결이 되어 마리이 쪽에서 마치 그 말을 엿듣기나 한 듯이 스스로 해결 짓게 된 것이 신통하다면 신통할까.
그라아샤에게는 숨은 만족을 주었을 반면에 삑톨에게는 얼마나 큰 상처를 주었을까는 추측하기에 넉넉하다. 주체스런 몸을 이끌고 휘돌아치는 삑톨의 양이 딱하기 짝없는 것이었다. 그러나 삑톨보다도 한층 속이 타는 것은 크리이긴이 아니었을까. 아키임과 같은 모습이기는 하나 신경질이요 빳빳스런 그의 기질이 애태우고 맞서던 사랑을 뺏기고 얼마나 속이 휘둘리웠을까. 말하는 법 없이 고함치는 법 없이 더욱 벙어리같이 침묵해 가는 그의 마음속이 얼마나 울가망하고 답답한 것이었을까. 가령 나는 그의 옆을 지나는 길에 무어라고 한마디쯤 말을 걸어보려는 것이나 첫째 그의 시선을 잡을 수가 없는 것이다. 눈앞을 보지 않고 그 어디인지 먼 데를 보고 있다. 그리고 그 노리고 있는 한 가지에 생각에 열중해 있음은 그 우악스런 눈매와 모가 져 보이는 턱의 각도로 짐작할 수 있다. 아마도 마리이일 듯한 그 한 가지 환영에 불같이 마음을 뺏기고 있는 것이다.



In [534]:
preprocessing_sentences = formal_preprocessing_text(source)
for preprocessing_sentence in preprocessing_sentences:
    print(preprocessing_sentence, end="\n\n")

그러나 자연스런 그들의 행위가 반면에 의외로 큰 희생을 요구했으니 그것은 단체에 끼치게 된 불리보다도 참으로 크리이긴과 삑톨 두 사람에게 던지게 된 불행이다.

삑톨이 조바심을 하고 안달을 하면서 두 사람의 종적을 찾으러 휘돌아치는 꼴에는 단의 책임자로서의 심정보다도 마리이에게 대한 실망과 초조가 드러나 보이는 듯하다.

며칠 전 호텔 병실에서 그의 아내 그라아샤가 마리이를 냉큼 내쫓아 달라고 고함을 쳤던 것이 그럴 필요조차 없게 제물에 해결이 되어 마리이 쪽에서 마치 그 말을 엿듣기나 한 듯이 스스로 해결 짓게 된 것이 신통하다면 신통할까.

그라아샤에게는 숨은 만족을 주었을 반면에 삑톨에게는 얼마나 큰 상처를 주었을까는 추측하기에 넉넉하다.

주체스런 몸을 이끌고 휘돌아치는 삑톨의 양이 딱하기 짝없는 것이었다.

그러나 삑톨보다도 한층 속이 타는 것은 크리이긴이 아니었을까.

아키임과 같은 모습이기는 하나 신경질이요 빳빳스런 그의 기질이 애태우고 맞서던 사랑을 뺏기고 얼마나 속이 휘둘리웠을까. 말하는 법 없이 고함치는 법 없이 더욱 벙어리같이 침묵해 가는 그의 마음속이 얼마나 울가망하고 답답한 것이었을까.

가령 나는 그의 옆을 지나는 길에 무어라고 한마디쯤 말을 걸어보려는 것이나 첫째 그의 시선을 잡을 수가 없는 것이다.

눈앞을 보지 않고 그 어디인지 먼 데를 보고 있다.

그리고 그 노리고 있는 한 가지에 생각에 열중해 있음은 그 우악스런 눈매와 모가 져 보이는 턱의 각도로 짐작할 수 있다.

아마도 마리이일 듯한 그 한 가지 환영에 불같이 마음을 뺏기고 있는 것이다.



### AIHUB 문서요약 텍스트

[Source](https://aihub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100&aihubDataSe=realm&dataSetSn=97)

In [535]:
path_list = ['AIHUB_문서요약 텍스트/Training/'+ '/**/*.json', 
             'AIHUB_문서요약 텍스트/Validation/'+ '/**/*.json']
train_file_name, valid_file_name = train_valid_json_file_name_list(path_list)

In [536]:
source_file = train_file_name[1]
print(source_file)

with open(source_file, 'r', encoding='utf-8') as one_json_file:
    one_json_sample = json.load(one_json_file)
    
source_df = pd.DataFrame(one_json_sample['documents'])
source_dict = dict(source_df['text'].explode())
source_json = pd.json_normalize(source_dict)  

source_list = []
for source_index in source_json:
    for source_nested_list in source_json[source_index]:
        for source_single_list in source_nested_list:
            source_sentence = ""
            for source_single in source_single_list:
                if type(source_single) == dict:
                    for key, value in source_single.items():
                        if key == "sentence":
                            source_sentence += " " + value
            if len(source_sentence) > 0:
                source_list.append(source_sentence)

AIHUB_문서요약 텍스트/Training\사설_train_original\train_original.json


In [537]:
print(source_list[1693])

 공정거래위원회가 롯데 현대 신세계 등 빅3 백화점이 총 1054개 중소 납품업체 판매수수료율을 3~7%포인트씩 인하하고 이달분부터 소급 적용하기로 했다고 밝혔다. 인하 대상은 3대 백화점에 납품하는 중소업체 총 2036개 중 51.8%에 해당한다. 지난 9월 6일 김동수 공정위원장이 유통업계 대표들을 불러 수수료 인하를 독려한 후 인하폭을 둘러싸고 공정위와 백화점 간에 팽팽한 신경전이 펼쳐졌으나 공정위가 가이드라인으로 제시한 범위에서 합의가 이뤄졌다.


In [538]:
preprocessing_sentences = formal_preprocessing_text(source_list[1693])
for preprocessing_sentence in preprocessing_sentences:
    print(preprocessing_sentence, end="\n\n")

공정거래위원회가 롯데 현대 신세계 등 빅3 백화점이 총 1054개 중소 납품업체 판매수수료율을 3 7 포인트씩 인하하고 이달분부터 소급 적용하기로 했다고 밝혔다.

인하 대상은 3대 백화점에 납품하는 중소업체 총 2036개 중 51.8 에 해당한다.

지난 9월 6일 김동수 공정위원장이 유통업계 대표들을 불러 수수료 인하를 독려한 후 인하폭을 둘러싸고 공정위와 백화점 간에 팽팽한 신경전이 펼쳐졌으나 공정위가 가이드라인으로 제시한 범위에서 합의가 이뤄졌다.



In [539]:
print(source_list[1687])

 중기 적합업종 제도가 소기의 성과를 내기 위해서는 대기업과 중소기업 간에 어느 정도 양해가 필요하다. 그런데 지난 1차 지정에서는 별 탈이 없었으나 이번 2차 지정에선 벌써부터 레미콘업계가 집단 반발 시위를 계획하는 등 말썽 소지가 벌어지고 있다. 동반성장위는 폼목을 늘리기 위해 졸속이 없었는지 살펴야 한다. 중소기업도 보호막에 안주하지 말고 위생, 안전성, 기술 품질, AS 등에서 소비자 만족도를 높이기 위한 노력을 배가해야 한다.


In [540]:
preprocessing_sentences = formal_preprocessing_text(source_list[1687])
for preprocessing_sentence in preprocessing_sentences:
    print(preprocessing_sentence, end="\n\n")

중기 적합업종 제도가 소기의 성과를 내기 위해서는 대기업과 중소기업 간에 어느 정도 양해가 필요하다.

그런데 지난 1차 지정에서는 별 탈이 없었으나 이번 2차 지정에선 벌써부터 레미콘업계가 집단 반발 시위를 계획하는 등 말썽 소지가 벌어지고 있다.

동반성장위는 폼목을 늘리기 위해 졸속이 없었는지 살펴야 한다.

중소기업도 보호막에 안주하지 말고 위생, 안전성, 기술 품질, AS 등에서 소비자 만족도를 높이기 위한 노력을 배가해야 한다.



In [541]:
source_file = train_file_name[2]
print(source_file)

with open(source_file, 'r', encoding='utf-8') as one_json_file:
    one_json_sample = json.load(one_json_file)
    
source_df = pd.DataFrame(one_json_sample['documents'])
source_dict = dict(source_df['text'].explode())
source_json = pd.json_normalize(source_dict)  

source_list = []
for source_index in source_json:
    for source_nested_list in source_json[source_index]:
        for source_single_list in source_nested_list:
            source_sentence = ""
            for source_single in source_single_list:
                if type(source_single) == dict:
                    for key, value in source_single.items():
                        if key == "sentence":
                            source_sentence += " " + value
            if len(source_sentence) > 0:
                source_list.append(source_sentence)

AIHUB_문서요약 텍스트/Training\신문기사_train_original\train_original.json


In [542]:
print(source_list[1899])

 체 씨는“언어부터 문화까지 모든 게 달라 한국 사회에 적응하는 것이 쉽지 않았는데 광양제철소 사내 바리스타로 근무하기 시작하면서부터 삶이 달라졌다”라며“광양지역에 안정적으로 적응할 수 있도록 따뜻한 도움의 손길을 내밀어 준 광양제철소 프렌즈봉사단에 고맙다”고 말했다.


In [543]:
preprocessing_sentences = formal_preprocessing_text(source_list[1899])
for preprocessing_sentence in preprocessing_sentences:
    print(preprocessing_sentence, end="\n\n")

체 씨는 언어부터 문화까지 모든 게 달라 한국 사회에 적응하는 것이 쉽지 않았는데 광양제철소 사내 바리스타로 근무하기 시작하면서부터 삶이 달라졌다 라며 광양지역에 안정적으로 적응할 수 있도록 따뜻한 도움의 손길을 내밀어 준 광양제철소 프렌즈봉사단에 고맙다 고 말했다.



### AIHUB 특허분야 자동분류 데이터

[Source](https://aihub.or.kr/aihubdata/data/view.do?currMenu=116&topMenu=100&aihubDataSe=ty&dataSetSn=547)

In [257]:
path_list = ['AIHUB_특허 분야 자동분류 데이터/Training/'+ '/**/*.json', 
             'AIHUB_특허 분야 자동분류 데이터/Validation/'+ '/**/*.json']
train_file_name, valid_file_name = train_valid_json_file_name_list(path_list)

In [258]:
source_file = train_file_name[2]
print(source_file)

with open(source_file, 'r', encoding='utf-8') as one_json_file:
    one_json_sample = json.load(one_json_file)

source_list = list(pd.DataFrame(one_json_sample['dataset'])['abstract'].dropna())

AIHUB_특허 분야 자동분류 데이터/Training\원천데이터\TS1\A_농업_임업및어업_01_03\01_농업\012_축산업\01220_양돈업.json


In [259]:
print(source_list[74])

목적 : 자동으로 양돈 사료를 공급하는 전자동 양돈 사료 공급 장치에 있어서, 양돈 체중의 변화에 따른 적절한 사료를 혼합하여 공급함으로써, 체중 변화에 따른 적절한 영양 공급 및 그로 인하여 사료비를 절감하고자 하는 것이다.구성 : 양돈사(20)의 외부에 설치된 2 또는 2 이상의 대형 사료 저장탱크(10)(10a); 호퍼(12)(12a); 구동부(30); 2개 또는 그 이상의 사료 저장탱크(10)(10a)에서 공급되는 사료를 혼합하기 위한 혼합조(40); 제 1 및 제 2사료 저장탱크(10)(10a)에서 혼합조(40)에 사료를 공급하기 위한 혼합 라인(13)(13a); 상기 혼합 라인(13)(13a)에 의해 혼합조(40)에 인입되는 사료의 양을 조절함으로써 혼합 비율을 조절하기 위한 제어부(50); 공급라인(14); 급이안내부(16); 급이통(22); 회수라인(18)으로 구성된 것이다.효과 : 사육되는 양돈의 성장 시기에 맞추어 적절한 사료의 공급을 자동적으로 할 수 있어, 효율적인 양돈 육성 관리 및 양돈의 성장 시기에 맞추어 값 비싼 사료의 양을 줄이고, 성장시기에 맞는 사료를 효율적으로 공급할 수 있어, 양돈 관리의 효율성 및 사료비의 절감을 가져올 수 있는 대단히 유용한 효과가 있는 것이다.


In [260]:
preprocessing_sentences = formal_preprocessing_text(source_list[74])
for preprocessing_sentence in preprocessing_sentences:
    print(preprocessing_sentence, end="\n\n")

목적 자동으로 양돈 사료를 공급하는 전자동 양돈 사료 공급 장치에 있어서, 양돈 체중의 변화에 따른 적절한 사료를 혼합하여 공급함으로써, 체중 변화에 따른 적절한 영양 공급 및 그로 인하여 사료비를 절감하고자 하는 것이다.

구성 양돈사(20)의 외부에 설치된 2 또는 2 이상의 대형 사료 저장탱크(10)(10a) 호퍼(12)(12a) 구동부(30) 2개 또는 그 이상의 사료 저장탱크(10)(10a)에서 공급되는 사료를 혼합하기 위한 혼합조(40) 제 1 및 제 2사료 저장탱크(10)(10a)에서 혼합조(40)에 사료를 공급하기 위한 혼합 라인(13)(13a) 상기 혼합 라인(13)(13a)에 의해 혼합조(40)에 인입되는 사료의 양을 조절함으로써 혼합 비율을 조절하기 위한 제어부(50) 공급라인(14) 급이안내부(16) 급이통(22) 회수라인(18)으로 구성된 것이다.

효과 사육되는 양돈의 성장 시기에 맞추어 적절한 사료의 공급을 자동적으로 할 수 있어, 효율적인 양돈 육성 관리 및 양돈의 성장 시기에 맞추어 값 비싼 사료의 양을 줄이고, 성장시기에 맞는 사료를 효율적으로 공급할 수 있어, 양돈 관리의 효율성 및 사료비의 절감을 가져올 수 있는 대단히 유용한 효과가 있는 것이다.



In [261]:
source_file = train_file_name[5]
print(source_file)

with open(source_file, 'r', encoding='utf-8') as one_json_file:
    one_json_sample = json.load(one_json_file)

source_list = list(pd.DataFrame(one_json_sample['dataset'])['abstract'].dropna())

AIHUB_특허 분야 자동분류 데이터/Training\원천데이터\TS1\A_농업_임업및어업_01_03\01_농업\012_축산업\01212_육우사육업.json


In [262]:
print(source_list[74])

본 고안은 농·축산 및 식품제조용 공급수에 미네랄이온을 생성 자동공급해 주는 제어장치로서 음용수 또는 제조수를 목적으로 공급되는 수돗물 및 지하수에 농축산 및 식품제조에 필요한 미네랄 워터로 충분한 량을 만들고 적절하게 공급하므로 내병성과 생산성을 높여주고 식품의 품질을 특화하는데 목적이 있다.이때, 이용되는 미네랄의 공급원은 점토광물로서 세리사이트, 맥반석, 무연프릿, 산성백토, 세피오라이트점토의 이상적인 조합으로 850도로 소성한 세라믹 볼을 스텐레스망에 담아서 미네랄 생성조 내의 사각대 위에 끼워놓고 미네랄 용출을 용이하게 하기위해 물에 운동량을 주고 제어하는 교반장치의 교반과 에어레이션의 물리적인 작용은 미네랄의 용출을 증가시킨다. 용해조안의 생성수는 Ca이온 센서로 감지되어 물의 출입을 통제하도록 하는 자동제어가 부착되며 음용수 및 살수라인에 원하는 농도의 미네랄 생성수를 자동 주입되도록 하는 장치이다.용해율 상승효과와 용존산소를 많게 하는 에어펌프가 하단부에 설치되며 생성된 높은 농도의 이온을 가진 미네랄생성수는 메인공급수에 주입되고 혼합된 처리수의 이온함량은 별도의 이온센서에 의해 측정된다. 이온 선택전극으로 감지된 농도는 미리 설정된 원하는 농도의 범위에 들고자 생성조에 희석보충수와 교반속도에 의해 스스로 이온함량을 조절제어하여 원하는 일정한 농도의 미네랄 워터가 공급하므로 가축사육 및 원예작물 생육에 필요한 무기영양분이 다량 함유된 미네랄 워터를 공급하고 식품제조에 있어 선도 유지 및 맛의 품질을 향상하도록 하는데 사용된다.


In [263]:
preprocessing_sentences = formal_preprocessing_text(source_list[74])
for preprocessing_sentence in preprocessing_sentences:
    print(preprocessing_sentence, end="\n\n")

본 고안은 농 축산 및 식품제조용 공급수에 미네랄이온을 생성 자동공급해 주는 제어장치로서 음용수 또는 제조수를 목적으로 공급되는 수돗물 및 지하수에 농축산 및 식품제조에 필요한 미네랄 워터로 충분한 량을 만들고 적절하게 공급하므로 내병성과 생산성을 높여주고 식품의 품질을 특화하는데 목적이 있다.

이때, 이용되는 미네랄의 공급원은 점토광물로서 세리사이트, 맥반석, 무연프릿, 산성백토, 세피오라이트점토의 이상적인 조합으로 850도로 소성한 세라믹 볼을 스텐레스망에 담아서 미네랄 생성조 내의 사각대 위에 끼워놓고 미네랄 용출을 용이하게 하기위해 물에 운동량을 주고 제어하는 교반장치의 교반과 에어레이션의 물리적인 작용은 미네랄의 용출을 증가시킨다.

용해조안의 생성수는 Ca이온 센서로 감지되어 물의 출입을 통제하도록 하는 자동제어가 부착되며 음용수 및 살수라인에 원하는 농도의 미네랄 생성수를 자동 주입되도록 하는 장치이다.

용해율 상승효과와 용존산소를 많게 하는 에어펌프가 하단부에 설치되며 생성된 높은 농도의 이온을 가진 미네랄생성수는 메인공급수에 주입되고 혼합된 처리수의 이온함량은 별도의 이온센서에 의해 측정된다.

이온 선택전극으로 감지된 농도는 미리 설정된 원하는 농도의 범위에 들고자 생성조에 희석보충수와 교반속도에 의해 스스로 이온함량을 조절제어하여 원하는 일정한 농도의 미네랄 워터가 공급하므로 가축사육 및 원예작물 생육에 필요한 무기영양분이 다량 함유된 미네랄 워터를 공급하고 식품제조에 있어 선도 유지 및 맛의 품질을 향상하도록 하는데 사용된다.



### AIHUB 뉴스기사 기계독해 데이터

[Source](https://aihub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100&aihubDataSe=realm&dataSetSn=577)

In [619]:
path_list = ['AIHUB_뉴스 기사 기계독해 데이터/Training/'+ '/**/*.json', 
             'AIHUB_뉴스 기사 기계독해 데이터/Validation/'+ '/**/*.json']
train_file_name, valid_file_name = train_valid_json_file_name_list(path_list)

In [620]:
source_file = train_file_name[0]
print(source_file)

with open(source_file, 'r', encoding='utf-8') as one_json_file:
    one_json_sample = json.load(one_json_file)

source_df = pd.DataFrame(one_json_sample['data'])
source_dict = dict(source_df['paragraphs'].explode())
source_json = pd.json_normalize(source_dict)
source_list = list(source_json.filter(regex='context').values[0])

AIHUB_뉴스 기사 기계독해 데이터/Training\원천데이터\TS_unanswerable.json


In [621]:
print(source_list[196])

지난해에 이어 올해도 충북지역 봄 축제가 줄줄이 취소될 전망이다. 일부 축제는 코로나19 확산을 막기 위해 차량에서 관람하는 드라이브 스루 방식으로 열리거나 비대면 축제로 방식을 바꾸는 등 변화도 시도하고 있다. 
 
제천시는 오는 4월 청풍문화마을 일원에서 개최할 예정이던 청풍호 벚꽃축제를 일찌감치 취소했다고 25일 밝혔다. 시는 코로나19 확산을 막기 위해 지난해에 이어 2년 연속 봄꽃 축제를 열지 않기로 했다. 
 
충주시는 민간사회단체 주도로 열던 충주호 벚꽃축제 개최 여부를 고심하고 있다. 지난해 코로나19 확산 여파로 축제는 열지 않았다. 
 
청주 도심을 가로지르는 무심천 벚꽃거리는 사회적 거리 두기가 적용될 전망이다. 청주시는 지난해 마스크 착용, 2m 이상 거리 두기, 주·정차 금지, 노점상 영업 금지, 음식물 취식 금지 등의 행정명령을 내렸다. 
 
일부 축제는 시기를 연기하거나 온라인 행사로 대체된다. 매년 5~6월 열던 세종대왕과 초정약수 축제는 오는 10월로 연기했다. 2003년부터 시작된 이 축제는 매년 청주시 청원구 내수읍 초정문화공원 일대에서 열린다. 세계 3대 광천수인 초정약수는 세종대왕이 눈병을 치료한 곳으로 유명하다. 지난해 5월 축제는 코로나19 사태로 취소됐다. 
 
단양군도 매년 4월 개최한 쌍둥이힐링페스티벌을 가을로 미뤘다. 2016년 국내 최초로 연 이 축제에는 해마다 전국 쌍둥이 참가자를 비롯해 수 만명의 관광객이 몰린다. 5월로 예정된 소백산 철쭉제는 개최 여부를 아직 결정하지 못했다. 
 
‘향수’의 시인 정지용을 기리는 문학축제 옥천 지용제도 10월로 연기했다. 
 
옥천군은 올해 옥천묘목축제를 취소하고 지난해처럼 온라인 판매행사로 대체했다. 묘목은 개별농원에서 정상적으로 구매할 수 있다. 군은 온라인 판매 활성화를 위해 군 홈페이지에 농원별 판매처를 게시하고 비대면 판매를 지원하고 있다. 
 
2021청주공예비엔날레는 오는 9월 온라인과 오프라인을 병행하는 형태로 치러진다. 행사 기간도 53일에서 40일로 단

In [622]:
preprocessing_sentences = formal_preprocessing_text(source_list[196])
for preprocessing_sentence in preprocessing_sentences:
    print(preprocessing_sentence, end="\n\n")

지난해에 이어 올해도 충북지역 봄 축제가 줄줄이 취소될 전망이다.

일부 축제는 코로나19 확산을 막기 위해 차량에서 관람하는 드라이브 스루 방식으로 열리거나 비대면 축제로 방식을 바꾸는 등 변화도 시도하고 있다.

제천시는 오는 4월 청풍문화마을 일원에서 개최할 예정이던 청풍호 벚꽃축제를 일찌감치 취소했다고 25일 밝혔다.

시는 코로나19 확산을 막기 위해 지난해에 이어 2년 연속 봄꽃 축제를 열지 않기로 했다.

충주시는 민간사회단체 주도로 열던 충주호 벚꽃축제 개최 여부를 고심하고 있다.

지난해 코로나19 확산 여파로 축제는 열지 않았다.

청주 도심을 가로지르는 무심천 벚꽃거리는 사회적 거리 두기가 적용될 전망이다.

청주시는 지난해 마스크 착용, 2m 이상 거리 두기, 주 정차 금지, 노점상 영업 금지, 음식물 취식 금지 등의 행정명령을 내렸다.

일부 축제는 시기를 연기하거나 온라인 행사로 대체된다.

매년 5 6월 열던 세종대왕과 초정약수 축제는 오는 10월로 연기했다.

2003년부터 시작된 이 축제는 매년 청주시 청원구 내수읍 초정문화공원 일대에서 열린다.

세계 3대 광천수인 초정약수는 세종대왕이 눈병을 치료한 곳으로 유명하다.

지난해 5월 축제는 코로나19 사태로 취소됐다.

단양군도 매년 4월 개최한 쌍둥이힐링페스티벌을 가을로 미뤘다.

2016년 국내 최초로 연 이 축제에는 해마다 전국 쌍둥이 참가자를 비롯해 수 만명의 관광객이 몰린다. 5월로 예정된 소백산 철쭉제는 개최 여부를 아직 결정하지 못했다.

옥천군은 올해 옥천묘목축제를 취소하고 지난해처럼 온라인 판매행사로 대체했다.

묘목은 개별농원에서 정상적으로 구매할 수 있다.

군은 온라인 판매 활성화를 위해 군 홈페이지에 농원별 판매처를 게시하고 비대면 판매를 지원하고 있다.

2021청주공예비엔날레는 오는 9월 온라인과 오프라인을 병행하는 형태로 치러진다.

오는 5월 예정된 진천 생거진천 농다리축제와 음성 품바축제는 3월 말까지 코로나19 확산 추이를 지켜본 뒤 개최 여부를

In [615]:
text = "포획이론의 본질은 정부규제의 부정적 효과에 대한 강조이다. 이 이론에 따르면, 규제기관은 일부 산업계의 이익에 포획되어 있으며, 무능하고(incompetent), 부패해서(corrupt) 사회 복지와 같은 공익을 위해 기여하지 못하고, 오히려 상황을 악화시키기 때문에 정부의 개입은 최소화되어야 한다는 것이다(Shleifer, 2005, p.440). 이러한 이유로 Stigler(1971)는 규제는 경제적으로 비효율을 초래할 뿐만 아니라 정치적으로도 정당화되기 어렵다는 점을 주장하였고, 같은 맥락에서 Peltzman(1976)은 이익집단들이 정치인들이나 규제기관을 통하여 그들의 이익에 맞는 규제를 수립하거나 지속시키고자 할 때 활용되는 자원은 사회적 손실(Social Deadweight Loss)을 야기하고, 국가 전체적으로도 자원의 낭비 혹은 왜곡된 자원배분을 가져온다고 보았다(최진욱, 2006, p.10에서 재인용)."
print(text)

포획이론의 본질은 정부규제의 부정적 효과에 대한 강조이다. 이 이론에 따르면, 규제기관은 일부 산업계의 이익에 포획되어 있으며, 무능하고(incompetent), 부패해서(corrupt) 사회 복지와 같은 공익을 위해 기여하지 못하고, 오히려 상황을 악화시키기 때문에 정부의 개입은 최소화되어야 한다는 것이다(Shleifer, 2005, p.440). 이러한 이유로 Stigler(1971)는 규제는 경제적으로 비효율을 초래할 뿐만 아니라 정치적으로도 정당화되기 어렵다는 점을 주장하였고, 같은 맥락에서 Peltzman(1976)은 이익집단들이 정치인들이나 규제기관을 통하여 그들의 이익에 맞는 규제를 수립하거나 지속시키고자 할 때 활용되는 자원은 사회적 손실(Social Deadweight Loss)을 야기하고, 국가 전체적으로도 자원의 낭비 혹은 왜곡된 자원배분을 가져온다고 보았다(최진욱, 2006, p.10에서 재인용).


In [616]:
preprocessing_sentences = formal_preprocessing_text(text)
for preprocessing_sentence in preprocessing_sentences:
    print(preprocessing_sentence, end="\n\n")

포획이론의 본질은 정부규제의 부정적 효과에 대한 강조이다.

이 이론에 따르면, 규제기관은 일부 산업계의 이익에 포획되어 있으며, 무능하고(incompetent), 부패해서(corrupt) 사회 복지와 같은 공익을 위해 기여하지 못하고, 오히려 상황을 악화시키기 때문에 정부의 개입은 최소화되어야 한다는 것이다.

이러한 이유로 Stigler(1971)는 규제는 경제적으로 비효율을 초래할 뿐만 아니라 정치적으로도 정당화되기 어렵다는 점을 주장하였고, 같은 맥락에서 Peltzman(1976)은 이익집단들이 정치인들이나 규제기관을 통하여 그들의 이익에 맞는 규제를 수립하거나 지속시키고자 할 때 활용되는 자원은 사회적 손실(Social Deadweight Loss)을 야기하고, 국가 전체적으로도 자원의 낭비 혹은 왜곡된 자원배분을 가져온다고 보았다.



### AIHUB 행정 문서 대상 기계독해 데이터

[Source](https://aihub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100&aihubDataSe=realm&dataSetSn=569)

In [272]:
path_list = ['AIHUB_행정 문서 대상 기계독해 데이터/Training/'+ '/**/*.json', 
             'AIHUB_행정 문서 대상 기계독해 데이터/Validation/'+ '/**/*.json']
train_file_name, valid_file_name = train_valid_json_file_name_list(path_list)

In [273]:
source_file = train_file_name[0]
print(source_file)

with open(source_file, 'r', encoding='utf-8') as one_json_file:
    one_json_sample = json.load(one_json_file) 

source_df = pd.DataFrame(one_json_sample['data'])
source_dict = dict(source_df['paragraphs'].explode())
source_json = pd.json_normalize(source_dict)
source_list = list(source_json.filter(regex='context').values[0])

AIHUB_행정 문서 대상 기계독해 데이터/Training\원천데이터\TS_tableqa.json


In [277]:
print(source_list[26])

1. 기획담당관-10318(2021.6.18.)호 및 공원여가과-2651(2021.6.21.)호, 공공개발기획단<br>   -5300(2021.6.22.)호와 관련입니다.<br>2. 귀 부서에서 신청하신 간행물 발간등록번호를 아래와 같이 통보해드리니, 안내에 따라 조치하신 후 간행물 납본결과를 작성하여 담당자 이메일로 제출해주시기 바랍니다.<br>  가. 간행물 발간등록번호 부여결과 <table><tbody><tr><td>신청부서</td><td>간행물명</td><td> 발간등록번호</td></tr><tr><td>기획조정실기획담당관</td><td>2021 하반기 달라지는 서울생활</td><td>51********-*******10</td></tr><tr><td>서부공원녹지사업소공원여가과</td><td>공원의 기억 월드컵공원(영문판)</td><td>51********-*******01</td></tr><tr><td>공공개발기획단</td><td>공공부지 전략거점개발계획수립</td><td>51********-*******01</td></tr></tbody></table> 간행물 발간등록번호 부여결과


In [278]:
preprocessing_sentences = tableqa_preprocessing_text(source_list[26])
for preprocessing_sentence in preprocessing_sentences:
    print(preprocessing_sentence, end="\n\n")

2. 귀 부서에서 신청하신 간행물 발간등록번호를 아래와 같이 통보해드리니, 안내에 따라 조치하신 후 간행물 납본결과를 작성하여 담당자 이메일로 제출해주시기 바랍니다.



In [279]:
source_file = train_file_name[3]
print(source_file)

with open(source_file, 'r', encoding='utf-8') as one_json_file:
    one_json_sample = json.load(one_json_file) 

source_df = pd.DataFrame(one_json_sample['data'])
source_dict = dict(source_df['paragraphs'].explode())
source_json = pd.json_normalize(source_dict)
source_list = list(source_json.filter(regex='context').values[0])

AIHUB_행정 문서 대상 기계독해 데이터/Training\원천데이터\TS_span_extraction.json


In [280]:
print(source_list[42])

제 목 복지건설위원장 의회운영업무추진비 지출
의회 복지건설위원장 의정활동 및 직무수행과 관련된 소요경비를 아래와 같이 지출하고자 합니다.
가. 지출내역 : 복지건설위원장 의정활동 및 직무수행과 관련된 소요경비 
나. 지출금액 : 금118,000원(금일십일만팔천원)
다. 지출일 : 2021. 4. 14.
라. 채주 : 하이난(주소 및 대표자 붙임 참조)
마. 지출방법 : 법인카드 이용대금 결제계좌로 입금
바. 예산과목 : 지방의회운영지원, 의정활동지원, 의정공통업무, 의회비, 의회운영업무추진비


In [283]:
preprocessing_sentences = formal_preprocessing_text(source_list[42])
for preprocessing_sentence in preprocessing_sentences:
    print(preprocessing_sentence, end="\n\n")

의회 복지건설위원장 의정활동 및 직무수행과 관련된 소요경비를 아래와 같이 지출하고자 합니다.



### AIHUB 기계독해

[Source](https://www.aihub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100&aihubDataSe=realm&dataSetSn=89)

In [290]:
path_list = ['AIHUB_기계독해'+ '/**/*.json']
file_name = json_file_name_list(path_list)

In [292]:
source_file = file_name[1]
print(source_file)

with open(source_file, 'r', encoding='utf-8') as one_json_file:
    one_json_sample = json.load(one_json_file)

source_df = pd.DataFrame(one_json_sample['data'])
source_dict = dict(source_df['paragraphs'].explode())
source_json = pd.json_normalize(source_dict)
source_list = list(source_json.filter(regex='context').values[0])

AIHUB_기계독해\ko_nia_clue0529_squad_all.json


In [293]:
print(source_list[55])

▲ 사진= KBS2 같이 살래요 영상캡처 ‘같이 살래요’가 시청률 30%를 목전에 두고 있다. 2일 시청률조사회사 닐슨코리아에 따르면 지난 1일 방송된 KBS2 주말드라마 ‘같이 살래요’는 전국 기준 27.7%를 기록했다. 이는 지난 방송분 21.8%에 비해 5.9% 포인트 상승한 수치다. 이는 같이 살래요의 자체최고 시청률이기도 하다. 이날 방송에서는 박효섭(유동근 분)의 빚으로 가족들이 위기를 맞았다. 또한 박효섭에게 복수하기 위해 상가거리 재개발을 진행했던 이미연(장미희 분)은 그간의 추억에 빠지며, 사업을 지속할지 궁금증을 자아냈다. 한편 동시간대 방송된 MBC 주말드라마 ‘부잣집 아들’은 4.3%, 11.2%, 9.9%, 10.5%를 나타내며 같이 살래요와 큰 격차를 보였다. 압도적인 상승세를 보인 같이 살래요가 시청률 30%까지 돌파할 수 있을지 귀추가 주목된다. /ekqls_star@fnnews.com fn스타 우다빈 기자


In [294]:
preprocessing_sentences = formal_preprocessing_text(source_list[55])
for preprocessing_sentence in preprocessing_sentences:
    print(preprocessing_sentence, end="\n\n")

2일 시청률조사회사 닐슨코리아에 따르면 지난 1일 방송된 KBS2 주말드라마 같이 살래요 는 전국 기준 27.7 를 기록했다.

이는 지난 방송분 21.8 에 비해 5.9 포인트 상승한 수치다.

이는 같이 살래요의 자체최고 시청률이기도 하다.

이날 방송에서는 박효섭(유동근 분)의 빚으로 가족들이 위기를 맞았다.

또한 박효섭에게 복수하기 위해 상가거리 재개발을 진행했던 이미연(장미희 분)은 그간의 추억에 빠지며, 사업을 지속할지 궁금증을 자아냈다.

한편 동시간대 방송된 MBC 주말드라마 부잣집 아들 은 4.3 , 11.2 , 9.9 , 10.5 를 나타내며 같이 살래요와 큰 격차를 보였다.

압도적인 상승세를 보인 같이 살래요가 시청률 30 까지 돌파할 수 있을지 귀추가 주목된다.



### AIHUB 도서자료 요약

[Source](https://aihub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100&aihubDataSe=realm&dataSetSn=93)

In [295]:
path_list = ['AIHUB_도서자료 요약/Training/'+ '/**/*.json', 
             'AIHUB_도서자료 요약/Validation/'+ '/**/*.json']
train_file_name, valid_file_name = train_valid_json_file_name_list(path_list)

In [320]:
source_file = train_file_name[100]
print(source_file)

with open(source_file, 'r', encoding='utf-8') as one_json_file:
    one_json_sample = json.load(one_json_file)

sentence_list = []
passage = one_json_sample["passage"]  
summary = one_json_sample["summary"]

passage_sentences = formal_preprocessing_text(passage)
for sentence in passage_sentences:
    sentence_list.append(sentence)   

summary_sentences = formal_preprocessing_text(summary)
for sentence in summary_sentences:
    sentence_list.append(sentence) 

AIHUB_도서자료 요약/Training\[원천]도서요약_train\기타\CNTS-00074217442_1.json


In [325]:
print(passage)

지구상의 장소(위도, 경도)가 다르면 지구자장의 방향과 세기도 달라진다. 또한 상당히 느릿한 변화이지만 시간적으로도 변화하고 있으며, 이것을 地磁氣 永年變化(geomagnetic secular variation)라고 부른다. 지자기는 3차원의 벡터량이며 수평면에 투영한 水平分力의 방위와 지리학적인 북(眞北)이 이루는 각인 偏角(declination)과 수평면으로부터의 경사각인 伏角(inclination)및, 벡터의 크기를 나타내는 全磁力(total force)의 3요소로 나타내는 경우가 많다. 지자기를 직접 관측하여 지구자장의 방향(편각 및 복각)을 알 수 있게 된 것은 영국 런던에서 16세기 후반에 시작된 연속적 관측이 가장 오래된 기록이며, 그것에서도 명확하게 지자기 영년변화가 보여진다. 그러나 그 이후 400년 이상의 관측결과를 가지고 볼 때는 그 변동은 그것 이상의 긴 주기인 모양으로 변동주기는 알려져 있지 않다.


In [326]:
print(summary)

지구상의 장소가 다르면 지구자장의 방향과 세기도 달라지며 시간적으로도 변화한다. 지자기를 가장 오래 관측한 것은 16세기 후반의 연속적 관측이며 이를 통해 지자기의 영년 변화를 볼 수 있었으나 이후 관측을 통해서도 변동 주기를 알 수는 없었다.


In [324]:
print(sentence_list)

['지구상의 장소(위도, 경도)가 다르면 지구자장의 방향과 세기도 달라진다.', '지자기는 3차원의 벡터량이며 수평면에 투영한 水平分力의 방위와 지리학적인 북(眞北)이 이루는 각인 偏角(declination)과 수평면으로부터의 경사각인 伏角(inclination)및, 벡터의 크기를 나타내는 全磁力(total force)의 3요소로 나타내는 경우가 많다.', '지자기를 직접 관측하여 지구자장의 방향(편각 및 복각)을 알 수 있게 된 것은 영국 런던에서 16세기 후반에 시작된 연속적 관측이 가장 오래된 기록이며, 그것에서도 명확하게 지자기 영년변화가 보여진다.', '그러나 그 이후 400년 이상의 관측결과를 가지고 볼 때는 그 변동은 그것 이상의 긴 주기인 모양으로 변동주기는 알려져 있지 않다.', '지구상의 장소가 다르면 지구자장의 방향과 세기도 달라지며 시간적으로도 변화한다.', '지자기를 가장 오래 관측한 것은 16세기 후반의 연속적 관측이며 이를 통해 지자기의 영년 변화를 볼 수 있었으나 이후 관측을 통해서도 변동 주기를 알 수는 없었다.']


### AIHUB 대규모 구매도서 기반 한국어 말뭉치 데이터

[Source](https://www.aihub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100&aihubDataSe=realm&dataSetSn=624)

In [327]:
path_list = ['AIHUB_대규모 구매도서 기반 한국어 말뭉치 데이터/sample/라벨링데이터/000/'+ '/**/*.json']
file_name = json_file_name_list(path_list)
for i in file_name:
    if 'INFO' in i:
        file_name.remove(i)

In [329]:
source_file = file_name[0]
print(source_file)

with open(file_name[0], 'r', encoding='utf-8') as one_json_file:
    one_json_sample = json.load(one_json_file)
    
source_df = pd.DataFrame(one_json_sample['paragraphs'])
source_dict = dict(source_df['sentences'].explode())
source_json = pd.json_normalize(source_dict)  

source_list = []
for source_index in source_json:
    for source_nested_list in source_json[source_index]:

        try:
            for source_single_list in source_nested_list:
                try:
                    for key, value in source_single_list.items():
                        if key == 'text':
                            source_list.append(value)
                except:
                    pass
        except:
            pass  

AIHUB_대규모 구매도서 기반 한국어 말뭉치 데이터/sample/라벨링데이터/000\BOOK_CORPUS_000_TEXT_0001.json


In [330]:
print(source_list[300])

새로 조, 경 단위의 부자가 된 사람들은 유기농 “공정거래” 인간 노동으로 만든 상품과 서비스에 엄청난 프리미엄을 지불할 능력이 될 것이다.


In [332]:
preprocessing_sentences = formal_preprocessing_text(source_list[300])
for preprocessing_sentence in preprocessing_sentences:
    print(preprocessing_sentence, end="\n\n")

새로 조, 경 단위의 부자가 된 사람들은 유기농 공정거래 인간 노동으로 만든 상품과 서비스에 엄청난 프리미엄을 지불할 능력이 될 것이다.



### AIHUB 도서자료 기계독해

[Source](https://aihub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100&aihubDataSe=realm&dataSetSn=92)

In [544]:
path_list = ['AIHUB_도서자료 기계독해/'+ '/**/*.json']
file_name = json_file_name_list(path_list)

In [545]:
source_file = file_name[0]
print(source_file)

with open(source_file, 'r', encoding='utf-8') as one_json_file:
    one_json_sample = json.load(one_json_file)

source_df = pd.DataFrame(one_json_sample['data'])
source_dict = dict(source_df['paragraphs'].explode())
source_json = pd.json_normalize(source_dict)
source_list = list(source_json.filter(regex='context').values[0])

source_filter_02 = source_json.filter(regex='[0123456789]').values[0]
source_df_02 = pd.DataFrame(source_filter_02)

for source_df_02_value in source_df_02.values:
    for value in source_df_02_value[0]:
        try:
            source_list.append(value['context'])
        except:
            pass

AIHUB_도서자료 기계독해\Validation\도서.json


In [546]:
print(source_list[50])

교통관련 빅데이터는 크게 공공재로 활용하기 위해 수집된 공공데이터와 민간에서 교통관련 부가사업을 하기 위해 수집한 민간데이터로 구분할 수 있다. 이 중 공공데이터는 개별 법률의 목적에 따라 관련 데이터의 범주를 획정하고, 경우에 따라 그 정보를 수집・보유할 수 있도록 한다. 경우에 따라서 일정 목적을 위하여 집대성한 데이터베이스 또는 시스템을 구축하여 민간부문에 제공할 수 있는 근거를 마련해 두고 있기도 하다. 가장 대표적인 『공공데이터의 제공 및 이용활성화의 관한 법률』로 동법은 2013년 7월 공포되어 10월 말 현재 시행되고 있으며, 대부분의 DB화된 교통정보는 당연히 공공데이터 법에 따른 공공데이터에 포함될 것이며, 여기서 나아가 제공대상 공공데이터에 해당하는 경우, 공공데이터 법에 따른 정보제공이 실질적으로 실현되는 객체가 된다. 물론 기존 교통정보 관련 법제에 따라 민간에 정보가 제공 또는 개방될 수도 있겠으나, 이때는 대부분 동일한 정보가 서로 다른 규범에 따라 대외적으로 개방・제공될 가능성이 높고, 제공청구권한을 가진 대상이 누군가에 따른 차이가 발생할 따름이다. 


In [547]:
preprocessing_sentences = formal_preprocessing_text(source_list[50])
for preprocessing_sentence in preprocessing_sentences:
    print(preprocessing_sentence, end="\n\n")

교통관련 빅데이터는 크게 공공재로 활용하기 위해 수집된 공공데이터와 민간에서 교통관련 부가사업을 하기 위해 수집한 민간데이터로 구분할 수 있다.

이 중 공공데이터는 개별 법률의 목적에 따라 관련 데이터의 범주를 획정하고, 경우에 따라 그 정보를 수집 보유할 수 있도록 한다.

경우에 따라서 일정 목적을 위하여 집대성한 데이터베이스 또는 시스템을 구축하여 민간부문에 제공할 수 있는 근거를 마련해 두고 있기도 하다.

가장 대표적인 공공데이터의 제공 및 이용활성화의 관한 법률 로 동법은 2013년 7월 공포되어 10월 말 현재 시행되고 있으며, 대부분의 DB화된 교통정보는 당연히 공공데이터 법에 따른 공공데이터에 포함될 것이며, 여기서 나아가 제공대상 공공데이터에 해당하는 경우, 공공데이터 법에 따른 정보제공이 실질적으로 실현되는 객체가 된다.

물론 기존 교통정보 관련 법제에 따라 민간에 정보가 제공 또는 개방될 수도 있겠으나, 이때는 대부분 동일한 정보가 서로 다른 규범에 따라 대외적으로 개방 제공될 가능성이 높고, 제공청구권한을 가진 대상이 누군가에 따른 차이가 발생할 따름이다.



### AIHUB 법률 규정 (판결서 약관 등) 텍스트 분석 데이터

[Source](https://aihub.or.kr/aihubdata/data/view.do?currMenu=116&topMenu=100&aihubDataSe=ty&dataSetSn=580)

In [548]:
path_list = ['AIHUB_법률 규정 (판결서 약관 등) 텍스트 분석 데이터/Training/' + '/**/*.json',
             'AIHUB_법률 규정 (판결서 약관 등) 텍스트 분석 데이터/Validation/' + '/**/*.json']
train_file_name, valid_file_name = train_valid_json_file_name_list(path_list)

In [549]:
source_file = train_file_name[6510]
print(source_file)

with open(source_file , 'r', encoding='utf-8') as one_json_file:
    one_json_sample = json.load(one_json_file)

    source_df = pd.DataFrame(one_json_sample)
    source_list = source_df.loc['disposalcontent']['disposal'] + \
        source_df.loc['rqestObjet']['mentionedItems'] + \
        source_df.loc['acusrAssrs']['assrs'] + \
        source_df.loc['dedatAssrs']['assrs'] + \
        source_df.loc['bsisFacts']['facts'] + \
        source_df.loc['courtDcss']['dcss'] + \
        source_df.loc['cnclsns']['close']      

AIHUB_법률 규정 (판결서 약관 등) 텍스트 분석 데이터/Training\라벨링데이터\1.Training\라벨링데이터\TL_1.판결문\01.민사\2018\2017브26.json


In [550]:
print(source_list[9])

청구인은, 0000 (주소 1 생략) ㅇ동 ㅁㅁㅁ호, ㅁㅁㅁ호, ㅁㅁㅁ호, 0000 (주소 2 생략) ㅁ동 ㅁㅁㅁ호 전세보증금 9,000만 원 반환채권, 0000 (주소 3 생략) ㅇㅇㅇ호는 소외 1이 소외 2, 소외 3, 소외 4, 소외 5에게 명의신탁한 재산이므로 이 부분도 재산분할대상에 포함되어야 한다고 주장한다.


In [551]:
preprocessing_sentences = formal_preprocessing_text(source_list[9])
for preprocessing_sentence in preprocessing_sentences:
    print(preprocessing_sentence, end="\n\n")

청구인은, 0000 (주소 1 생략) ㅇ동 ㅁㅁㅁ호, ㅁㅁㅁ호, ㅁㅁㅁ호, 0000 (주소 2 생략) ㅁ동 ㅁㅁㅁ호 전세보증금 9,000만 원 반환채권, 0000 (주소 3 생략) ㅇㅇㅇ호는 소외 1이 소외 2, 소외 3, 소외 4, 소외 5에게 명의신탁한 재산이므로 이 부분도 재산분할대상에 포함되어야 한다고 주장한다.



In [552]:
source_file = train_file_name[0]
print(source_file)

with open(source_file, 'r', encoding='utf-8') as one_json_file:
    one_json_sample = json.load(one_json_file)

source_df = pd.DataFrame.from_dict(one_json_sample, orient='index')
source_df = source_df.transpose()
source_df = pd.DataFrame(one_json_sample)
source_list = [source_df['clauseArticle'][0], source_df['comProvision'][0]]

AIHUB_법률 규정 (판결서 약관 등) 텍스트 분석 데이터/Training\라벨링데이터\1.Training\라벨링데이터\TL_2.약관\01.유리\001_보증_가공.json


In [553]:
print(source_list[0])

제2조(보증금액)
 ① 이 보증서에 의한 보증금액은 채권자의 채무자에 대한 보증부대출 예정금액에 보증비율을 곱한 금액으로 합니다.


In [554]:
preprocessing_sentences = formal_preprocessing_text(source_list[0])
for preprocessing_sentence in preprocessing_sentences:
    print(preprocessing_sentence, end="\n\n")

In [555]:
source_file = train_file_name[900]
print(source_file)

with open(source_file, 'r', encoding='utf-8') as one_json_file:
    one_json_sample = json.load(one_json_file)

source_df = pd.DataFrame.from_dict(one_json_sample, orient='index')
source_df = source_df.transpose()
source_list = [source_df['clauseArticle'][0][0], source_df['illdcssBasiss'][0][0],
                source_df['relateLaword'][0][0]]   

AIHUB_법률 규정 (판결서 약관 등) 텍스트 분석 데이터/Training\라벨링데이터\1.Training\라벨링데이터\TL_2.약관\02.불리\014_예식업_가공.json


In [556]:
print(source_list[2])

제9조(계약의 해제,해지) 
계약의 해제,해지에 관하여 정하고 있는 약관의 내용 중 다음 각 호의 어느 하나에 해당되는 내용을 정하고 있는 조항은 무효로 한다.
  3. 법률에 따른 사업자의 해제권 또는 해지권의 행사 요건을 완화하여 고객에게 부당하게 불이익을 줄 우려가 있는 조항


In [557]:
preprocessing_sentences = formal_preprocessing_text(source_list[2])
for preprocessing_sentence in preprocessing_sentences:
    print(preprocessing_sentence, end="\n\n")

계약의 해제,해지에 관하여 정하고 있는 약관의 내용 중 다음 각 호의 어느 하나에 해당되는 내용을 정하고 있는 조항은 무효로 한다.



### AIHUB 일반상식

[Source](https://aihub.or.kr/aihubdata/data/view.do?currMenu=116&topMenu=100&aihubDataSe=ty&dataSetSn=106)

In [558]:
path_list = ['AIHUB_일반상식/'+ '/**/*.json']
file_name_list = json_file_name_list(path_list)

In [559]:
for file_name in file_name_list:
    if 'ko_wiki_v1_squad' in file_name:
        source_file = file_name

In [560]:
print(source_file)

with open(source_file, 'r', encoding='utf-8') as one_json_file:
    one_json_sample = json.load(one_json_file)

source_df = pd.DataFrame(one_json_sample['data'])
source_dict = dict(source_df['paragraphs'].explode())
source_json = pd.json_normalize(source_dict)
source_list = list(source_json.filter(regex='context').values[0])

AIHUB_일반상식\ko_wiki_v1_squad.json


In [561]:
print(source_list[9961])

2014년 11월 현재, 점포 수는 19개 나라, 미국 27개 주에 걸쳐 250개이다. 핑크베리의 점포는 대부분 캘리포니아 주 남부에 있다. 뉴욕 시에도 16개의 점포가 있다. 핑크베리의 첫 번째 점포는 2005년 1월 문을 열었다. 한국계 미국인 황혜경 이 창업하였다. 일부 사람들은 핑크베리의 제품들을 가리켜 "크랙베리"(Crackberry), "얼린 헤로인 주스"(frozen heroin juice)라고 불렀다. (크랙베리라고 별명이 붙은 것에는 또한 유명한 휴대전화 블랙베리가 있다.) 핑크베리사는 그 회사의 고객들이 "컬트" 취향인 것을 잘 인지하고서는, 회사 홈페이지에 "그루피 코너"(groupie corner)라는 것을 두고 있다.


In [562]:
preprocessing_sentences = formal_preprocessing_text(source_list[9961])
for preprocessing_sentence in preprocessing_sentences:
    print(preprocessing_sentence, end="\n\n")

2014년 11월 현재, 점포 수는 19개 나라, 미국 27개 주에 걸쳐 250개이다.

핑크베리의 점포는 대부분 캘리포니아 주 남부에 있다.

핑크베리의 첫 번째 점포는 2005년 1월 문을 열었다.

일부 사람들은 핑크베리의 제품들을 가리켜 크랙베리 (Crackberry), 얼린 헤로인 주스 (frozen heroin juice)라고 불렀다.

크랙베리라고 별명이 붙은 것에는 또한 유명한 휴대전화 블랙베리가 있다.

핑크베리사는 그 회사의 고객들이 컬트 취향인 것을 잘 인지하고서는, 회사 홈페이지에 그루피 코너 (groupie corner)라는 것을 두고 있다.



In [563]:
print(source_list[619])

〈밤 어딘가에서 ~night shift~〉(일본어: 夜のどこかで 〜night shift〜 요루노도코카데 나이트시프트)는 일본의 여가수 나카모리 아키나(中森明菜)의 29번째 싱글로 1994년 9월 2일에 발매하였다. (8cmCD: MVDD-10007) 작사는 나츠노 세리코(夏野芹子)가, 작곡과 편곡은 고토 츠구토시(後藤次利)가 담당하였고 MCA 빅터의 레이블로 출시되었다. B사이드 곡은 〈Rose Bud〉로 A사이드와 마찬가지로 작사는 나츠노 세리코가, 작곡과 편곡은 역시 고토 츠구토시가 도맡았다. 이 곡은 뉴스 프로그램인 《NNN 오늘의 사건》(NNNきょうの出来事)의 엔딩곡으로 제작되었는데 디스크 재킷 역시 뉴스 프로그램의 앵커인 사쿠라이 요시코와 촬영하였다. 나카모리는 뉴스 프로그램의 주제가로 사용될 것이라는 말을 듣고 어떤 목소리로 노래해야 할지 고민했지만 뉴스에 등장하는 여성들을 향한 응원가로 생각하면서 불렀다고 밝혔다. 1994년 9월 12일, 오리콘 주간 싱글 차트 14위로 처음 등장하여 차트 톱100을 총 7주간 지켰다.


In [564]:
preprocessing_sentences = formal_preprocessing_text(source_list[619])
for preprocessing_sentence in preprocessing_sentences:
    print(preprocessing_sentence, end="\n\n")

B사이드 곡은 Rose Bud 로 A사이드와 마찬가지로 작사는 나츠노 세리코가, 작곡과 편곡은 역시 고토 츠구토시가 도맡았다.

이 곡은 뉴스 프로그램인 NNN 오늘의 사건 (NNN 出来事)의 엔딩곡으로 제작되었는데 디스크 재킷 역시 뉴스 프로그램의 앵커인 사쿠라이 요시코와 촬영하였다.

나카모리는 뉴스 프로그램의 주제가로 사용될 것이라는 말을 듣고 어떤 목소리로 노래해야 할지 고민했지만 뉴스에 등장하는 여성들을 향한 응원가로 생각하면서 불렀다고 밝혔다.

1994년 9월 12일, 오리콘 주간 싱글 차트 14위로 처음 등장하여 차트 톱100을 총 7주간 지켰다.

